まずは普通のモデル  
レイヤーの幅を少し大きくした

特徴セット  
・prod  
・stats  
・pca(80, 10)  
・kmeans(22, 5)  
・

In [1]:
import warnings, random, os, sys, tqdm, time
sys.path.append("../")
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, RobustScaler

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import KFold
from scipy.spatial import distance
from scipy.stats import pearsonr

import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.modules.loss import _WeightedLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau

from pytorch_tabnet.tab_model import TabNetRegressor
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

pd.set_option("display.max_columns", 1200)
pd.set_option("display.max_rows", 1200)
%matplotlib inline



In [2]:
def metric(y_true, y_pred):
    res = []
    for i in range(0, y_true.shape[1]):
        y = y_true[:,i]
        pred = y_pred[:,i]
        res.append(log_loss(y, pred))
    return np.mean(res)

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True
seed_everything(42)
        
    
def make_scaler(flag, seed):
    if flag == "quantile":
        return QuantileTransformer(n_quantiles=100,random_state=seed, output_distribution="normal")
    elif flag == "gauss":
        return GaussRankScaler()
    elif flag == "standard":
        return StandardScaler()
    elif flag == "minmax":
        return MinMaxScaler()
    elif flag == "robust":
        return RobustScaler()
    
def special_pearsonr(X, Y):
    pearsonr_value = pearsonr(X, Y)
    if pearsonr_value[0] < 0:
        return abs(pearsonr_value[0])
    else:
        return 1 - pearsonr_value[0]    
    
def extract(df1, df2):
    size = df1.shape[0]
    df = df1.append(df2).reset_index(drop=True)
    li = [[('48 * D1',[-0.52545,-0.17565,0.47840000000000005,0.5815,-0.31865,0.17855,-1.777,-0.5191,0.49124999999999996,-0.16135],[0.0006363961030679012,0.0009192388155425283,0.0009899494936611753,0.0015556349186103902,0.0017677669529663704,0.002333452377915605,0.0028284271247461927,0.0033941125496953686,0.003747665940288721,0.004454772721475249],['g-645','g-130','g-328','c-5','c-95','g-551','c-3','g-487','g-578','g-702'])],[('48 * D1',[-2.1,-0.8259,-0.19345,-2.725,-1.0545,-0.032850000000000004,0.9117500000000001,-0.22935,0.07025,-1.2934999999999999],[0.0014142135623729393,0.0014142135623730963,0.002333452377915605,0.004242640687119446,0.004949747468305916,0.005727564927611037,0.007000357133746756,0.0071417784899841285,0.007424621202458746,0.007778174593051951],['c-90','g-10','g-641','c-41','g-76','g-422','g-311','g-658','g-180','c-95']),('24 * D1',[0.22485,0.26115,0.10345,-0.1796,-0.4128,-0.53035,-0.7067000000000001,0.0696,-1.167,-0.25465],[0.0007778174593051951,0.0009192388155424891,0.0013435028842544493,0.0025455844122715655,0.002545584412271605,0.0031819805153394274,0.0035355339059327407,0.004384062043356593,0.005656854249492385,0.005868986283848326],['g-562','g-15','g-548','g-104','g-626','g-78','g-448','g-488','g-459','c-17']),('24 * D2',[-0.36014999999999997,0.0932,-0.24095,0.6064,-0.10364999999999999,-0.40195000000000003,0.1689,0.4152,0.07189999999999999,-0.8058000000000001],[0.000494974746830568,0.001131370849898479,0.0013435028842544493,0.0028284271247461927,0.0028991378028648492,0.0034648232278140546,0.005091168824543131,0.005374011537017758,0.005798275605729689,0.006646803743153521],['g-112','g-237','g-158','g-556','g-603','g-655','g-300','g-255','g-725','g-267']),('72 * D2',[-0.19765,0.3841,0.14284999999999998,0.5043,0.15460000000000002,0.17245,-0.21,0.4841,-0.004,-0.58005],[0.0003535533905932741,0.0004242640687119603,0.0012020815280171357,0.0022627416997969383,0.004242640687119269,0.0045961940777125634,0.00466690475583121,0.005656854249492346,0.006081118318204309,0.008980256121069185],['g-314','g-210','g-310','c-70','g-551','c-29','g-695','c-72','g-87','c-43'])],[('24 * D1',[0.039099999999999996,0.3534,0.03325,0.1378,-0.79235,-0.7769,-0.78105,0.28805000000000003,0.15344999999999998,0.4949],[0.00042426406871192595,0.0007071067811865482,0.0013435028842544395,0.0016970562748477233,0.001909188309203625,0.0026870057685088986,0.0072831998462214225,0.00756604255869605,0.008414570696119911,0.010182337649086262],['g-693','g-721','c-44','g-435','c-89','g-673','g-125','g-555','g-14','g-222'])],[('48 * D1',[0.0567,0.33335000000000004,0.27055,-0.33565,0.40115,-0.49029999999999996,-0.29885,0.5205500000000001,0.69155,-0.03595],[0.00014142135623731355,0.00021213203435598016,0.0007778174593051951,0.0009192388155424891,0.0016263455967290372,0.0016970562748477233,0.0034648232278140937,0.003747665940288682,0.0045961940777125235,0.005161879502661798],['g-131','c-1','g-51','g-155','g-538','g-556','g-58','g-311','g-234','g-751'])],[('48 * D2',[-0.4992,0.15375,0.0053,-0.47945000000000004,0.30825,-0.2615,-0.0992,-0.11524999999999999,-0.7787,0.1287],[0.0002828427124746271,0.0013435028842544493,0.0021213203435596424,0.0023334523779156243,0.0023334523779156243,0.003676955262170035,0.004242640687119289,0.005020458146424494,0.005374011537017719,0.008626702730475881],['g-583','g-424','g-347','g-172','g-463','g-281','g-692','g-422','g-224','g-75'])],[('48 * D1',[-0.20005,-0.1312,-1.0699999999999998,0.62425,-0.79655,-0.55875,0.9319999999999999,-0.7105,-1.1475,0.195],[0.0012020815280171357,0.0014142135623730963,0.0014142135623730963,0.0027577164466275456,0.0044547727214752295,0.0044547727214752295,0.004666904755831249,0.005656854249492385,0.006363961030678855,0.006505382386916227],['g-253','g-288','g-540','g-425','g-232','g-684','g-636','g-670','g-30','g-756'])],[('48 * D1',[0.3942,-2.542,0.49785,-1.9725000000000001,0.8365,-0.7081500000000001,-0.3826,0.8311999999999999,0.81025,-3.6799999999999997],[0.0008485281374238421,0.0014142135623732533,0.0017677669529663704,0.002121320343559566,0.002545584412271605,0.002616295090390173,0.0033941125496954466,0.004101219330881916,0.004171930009000642,0.004242640687119132],['g-220','g-603','g-630','g-8','g-422','g-580','g-268','g-703','g-112','g-114'])],[('24 * D1',[-0.46904999999999997,-0.7054,0.259,-0.33825,-0.6555,-0.42605,0.3509,-0.020900000000000002,0.12004999999999999,0.4853],[0.000494974746830568,0.000989949493661136,0.0014142135623730963,0.0014849242404917434,0.0026870057685088986,0.0034648232278140937,0.0035355339059327407,0.004101219330881978,0.004737615433949867,0.006505382386916227],['g-515','g-76','g-213','g-652','g-299','g-469','g-272','g-501','g-132','g-500']),('72 * D2',[0.9716,-0.41015,-0.1483,-0.21115,-0.03075,0.23385,-0.585,0.3595,-0.8644000000000001,-0.22665000000000002],[0.0008485281374238421,0.0009192388155425283,0.0016970562748477038,0.0017677669529663704,0.002192031021678299,0.0026162950903902125,0.0026870057685088986,0.0035355339059327407,0.004101219330881916,0.004171930009000622],['g-420','g-536','g-241','g-345','c-85','g-177','g-32','g-233','g-221','g-444'])],[('48 * D2',[-1.7035,-0.7546999999999999,1.3900000000000001,-0.5546,-2.857,-0.44830000000000003,1.581,-0.90705,-0.23504999999999998,-2.5294999999999996],[0.0007071067811864697,0.0008485281374238421,0.0014142135623730963,0.0026870057685088986,0.0028284271247461927,0.0032526911934581135,0.004242640687119289,0.0045961940777125235,0.005161879502661798,0.0106066017177983],['c-62','g-196','g-683','g-112','g-598','g-302','g-646','g-28','g-440','g-586']),('72 * D1',[-3.884,-2.8945,-2.9055,-0.43574999999999997,-5.026,8.1515,-9.8135,0.60815,4.6085,-3.9305000000000003],[0.0014142135623729393,0.002121320343559723,0.002121320343559723,0.0023334523779156243,0.005656854249492385,0.006363961030679169,0.006363961030679169,0.013930003589374943,0.016263455967290372,0.016263455967290685],['g-166','g-696','g-479','g-56','g-534','g-712','c-75','g-538','g-663','g-185']),('72 * D2',[1.7814999999999999,-2.8245,1.002,-1.1885,-4.025,0.7113,0.79455,0.2023,3.5685000000000002,-0.00185],[0.0007071067811866267,0.002121320343559723,0.0028284271247461927,0.003535533905932819,0.005656854249492385,0.006081118318204346,0.007000357133746834,0.007919595949289324,0.009192388155425361,0.00954594154601839],['g-206','c-85','g-576','g-738','c-83','g-130','g-513','g-573','g-724','g-495']),('24 * D1',[-0.83255,-2.1595,-2.207,0.1578,-0.13885,-1.217,1.4385,-0.9796,1.377,0.1476],[0.001767766952966331,0.004949747468305602,0.005656854249492385,0.00622253967444162,0.006858935777509501,0.007071067811865481,0.007778174593051951,0.008485281374238578,0.008485281374238578,0.008909545442950499],['g-540','g-214','g-189','g-756','g-765','g-581','g-193','g-293','g-2','g-463']),('24 * D2',[0.26554999999999995,1.169,-1.1235,-1.8245,3.683,0.25475000000000003,-2.2969999999999997,-8.7895,-1.6909999999999998,-3.811],[0.0010606601717798223,0.0028284271247461927,0.004949747468305759,0.006363961030678855,0.007071067811865638,0.007424621202458756,0.008485281374238578,0.009192388155425047,0.009899494936611674,0.009899494936611831],['g-358','g-33','g-770','g-230','g-91','g-479','g-156','g-37','c-77','g-735']),('48 * D1',[-4.0015,1.7865,1.1829999999999998,0.47765,0.19635,0.18380000000000002,1.5755,-0.55555,-5.153499999999999,1.1705],[0.0007071067811861557,0.003535533905932662,0.007071067811865481,0.00756604255869605,0.009545941546018401,0.011879393923934006,0.012020815280171397,0.01265721138323922,0.014849242404917433,0.01484924240491759],['g-61','g-547','g-413','g-617','g-168','g-171','g-558','g-21','g-165','g-268'])],[('24 * D1',[0.71875,-0.37165000000000004,-0.4713,-0.33545,0.54285,0.0605,0.042050000000000004,0.12735,-0.24335,-0.1121],[0.00035355339059323483,0.0010606601717798223,0.0011313708498985084,0.0014849242404917826,0.0014849242404918216,0.001555634918610405,0.00233345237791561,0.004171930009000632,0.0045961940777125634,0.00466690475583122],['g-143','g-88','g-543','g-599','g-577','g-278','g-293','g-432','g-256','g-456']),('72 * D1',[-0.6351,-0.0944,-0.5325500000000001,0.4333,-0.0691,-0.0253,0.54305,0.5698000000000001,0.49165000000000003,-0.6429499999999999],[0.00042426406871188177,0.0008485281374238617,0.001060660171779783,0.0015556349186104294,0.0016970562748477138,0.002687005768508879,0.0028991378028648397,0.0035355339059327407,0.004030508652763309,0.00487903679018719],['g-311','g-379','g-595','g-149','g-66','g-136','g-140','g-250','g-292','g-181']),('72 * D2',[0.18215,0.4544,0.09375,-0.84195,0.3671,-0.00395,0.12165000000000001,0.49715,-0.3963,-0.18764999999999998],[7.071067811864696e-05,0.0007071067811865482,0.0010606601717798223,0.0012020815280171554,0.0014142135623730963,0.00148492424049175,0.0023334523779156148,0.0026162950903902125,0.005374011537017758,0.005586143571373719],['c-46','g-429','g-548','g-333','g-372','g-320','g-507','c-92','g-591','g-560'])],[('24 * D2',[-0.21195,-0.09415000000000001,0.21915,0.3353,0.25329999999999997,0.1876,-0.20224999999999999,0.41315,-0.3426,0.08715],[7.071067811864696e-05,0.0014849242404917531,0.0017677669529663704,0.0022627416997969383,0.0029698484809835258,0.00622253967444162,0.0064346717087976,0.006576093065034913,0.00664680374315356,0.007566042558696059],['g-515','g-511','g-420','g-36','g-719','c-47','g-313','g-305','g-712','g-80']),('24 * D1',[0.1567,-0.61435,0.49935,0.2333,0.0017,0.57535,-0.13175,0.17525000000000002,-0.16225,0.3957],[0.0002828427124746271,0.00035355339059331333,0.0017677669529664096,0.002262741699796958,0.0024041630560342614,0.0026162950903902515,0.0031819805153394665,0.0036062445840514073,0.005444722215136405,0.005515432893255091],['g-182','g-447','g-526','g-251','g-331','g-290','g-205','g-435','g-60','g-340']),('48 * D2',[-0.1513,0.1958,-1.4009999999999998,-0.1163,-0.11925,-0.23725000000000002,-0.40515,-0.15205000000000002,-0.33089999999999997,0.1089],[0.00014142135623731355,0.0009899494936611753,0.0014142135623730963,0.002262741699796958,0.0037476659402887014,0.004171930009000622,0.004171930009000642,0.004454772721475249,0.005374011537017758,0.005656854249492385],['g-59','g-88','g-17','g-254','g-366','g-378','g-446','g-706','g-26','g-55']),('72 * D1',[0.3342,0.81525,0.40175,0.6895,-0.9074,-0.8903,-0.6224000000000001,-0.33245,-1.2405,0.44810000000000005],[0.0004242640687119603,0.0016263455967290372,0.0019091883092037035,0.0021213203435596446,0.0026870057685088986,0.0029698484809834867,0.005374011537017719,0.006293250352560247,0.006363961030679012,0.006788225099390855],['c-88','g-707','g-27','g-263','g-693','g-316','g-637','c-32','g-612','g-280']),('72 * D2',[0.31025,-0.11715,-0.35865,-1.1345,0.1875,-0.1585,-0.16620000000000001,0.56115,0.37424999999999997,0.03485],[7.071067811868621e-05,0.00021213203435596054,0.000494974746830568,0.002121320343559566,0.0026870057685088986,0.003818376618407368,0.004808326112068524,0.005161879502661778,0.009828784258492988,0.010111626970967631],['c-43','g-557','g-523','g-157','g-766','g-538','g-155','g-568','g-353','c-86'])],[('48 * D2',[-0.25975000000000004,0.2306,0.007299999999999999,-0.06705,-0.0932,0.17585,0.6604,0.47165,-0.13465,-0.57645],[0.0003535533905932741,0.0007071067811865482,0.0015556349186104045,0.003040559159102153,0.0036769552621700443,0.003889087296526015,0.003959797974644701,0.0045961940777125634,0.005303300858899111,0.005444722215136444],['g-723','c-41','g-678','g-134','g-494','g-322','g-725','g-306','g-299','g-475']),('72 * D1',[0.6018,0.031,-0.9779500000000001,0.7201,-0.14579999999999999,-0.0398,0.314,0.2733,-0.13095,0.7091000000000001],[0.0005656854249491757,0.0005656854249492371,0.0007778174593051951,0.003252691193458153,0.0057982756057296985,0.0076367532368147115,0.009050966799187833,0.009899494936611674,0.010394469683442252,0.010606601717798222],['g-663','g-631','g-339','g-690','g-116','g-123','g-525','g-152','g-386','g-442']),('24 * D2',[-0.19519999999999998,0.15375,0.10064999999999999,0.20325,-0.7637,0.4314,-0.30835,-0.10919999999999999,-0.109,0.0298],[0.00014142135623731355,0.000494974746830568,0.0020506096654409876,0.004030508652763328,0.004949747468305837,0.004949747468305837,0.005586143571373738,0.005656854249492376,0.0059396969619669926,0.006081118318204309],['g-57','g-307','g-19','g-747','g-22','g-56','g-323','g-33','c-30','g-356']),('72 * D2',[0.05235,0.49060000000000004,1.0465,-0.27005,0.3594,0.1273,0.44625000000000004,-1.0645,0.1526,-0.9944],[0.00021213203435596054,0.0005656854249492149,0.0007071067811864697,0.0009192388155425283,0.0035355339059327407,0.004101219330881986,0.0044547727214752685,0.004949747468305916,0.007071067811865462,0.0072124891681028536],['g-69','g-109','g-574','g-570','g-249','g-569','g-375','g-254','g-635','g-177'])],[('48 * D1',[0.34545000000000003,0.3873,-0.49775,0.3255,-0.08065,0.07830000000000001,-0.36560000000000004,0.3747,0.38115,-0.0548],[0.00021213203435598016,0.0007071067811865482,0.0012020815280171162,0.0014142135623730963,0.001909188309203684,0.0022627416997969482,0.006081118318204307,0.00777817459305203,0.008273149339882598,0.011879393923934006],['g-485','g-734','g-552','g-442','g-347','g-398','g-264','g-620','c-83','g-455']),('24 * D1',[0.5547,0.22405,0.64735,-0.9711000000000001,0.68235,-0.37675000000000003,-0.04175,-0.6192500000000001,0.4365,1.395],[0.00028284271247458785,0.0017677669529663704,0.0028991378028648397,0.0035355339059327407,0.00601040764008562,0.007283199846221462,0.007848885271170676,0.007848885271170676,0.008485281374238578,0.008485281374238578],['g-506','g-528','g-375','g-68','g-426','g-259','g-591','g-731','c-48','g-138']),('48 * D2',[-0.8604,-0.42885,-0.6743,0.8705499999999999,0.99935,-0.6110500000000001,-0.42410000000000003,0.671,-0.1654,-0.6643],[0.00014142135623737242,0.00021213203435594088,0.00028284271247458785,0.0004949747468306073,0.0006363961030679012,0.0007778174593051951,0.001979898987322311,0.0025455844122715264,0.0025455844122715655,0.0028284271247461927],['g-736','g-762','g-213','c-67','g-40','g-723','g-105','c-90','g-578','g-255'])],[('24 * D1',[0.03955,0.29675,0.22139999999999999,0.29945,-0.21860000000000002,1.249,-0.25585,-0.44984999999999997,-0.5482,-0.60345],[0.00021213203435596541,0.0007778174593051951,0.0014142135623730766,0.0030405591591021333,0.004242640687119269,0.004242640687119289,0.0045961940777125634,0.0060104076400856596,0.006646803743153521,0.006717514421272168],['g-348','g-206','g-546','c-34','g-404','g-50','c-88','g-152','g-85','g-125']),('48 * D1',[0.33799999999999997,0.323,-0.10185,-0.3651,-0.7888,-0.012650000000000002,-0.6154999999999999,-0.17609999999999998,0.018750000000000003,-0.09405],[0.0011313708498984691,0.0012727922061358025,0.001909188309203674,0.001979898987322311,0.0022627416997969383,0.002616295090390226,0.004101219330881916,0.005091168824543151,0.006434671708797583,0.009687362902255704],['g-301','c-8','g-135','g-400','g-317','c-11','g-416','g-107','g-701','g-207'])],[('72 * D2',[-0.33165,1.448,-0.49355000000000004,-0.6557,0.7861499999999999,0.07885,-0.1336,0.51615,-0.32885,-0.2141],[0.00021213203435594088,0.0028284271247461927,0.003181980515339467,0.004384062043356661,0.005161879502661778,0.005161879502661798,0.005374011537017758,0.00601040764008562,0.006717514421272207,0.007636753236814716],['g-611','c-10','g-102','g-381','c-65','c-23','g-263','c-83','g-363','g-723']),('48 * D2',[-0.30645,-0.17365,-0.30425,0.13629999999999998,0.3067,-0.8453999999999999,-0.27575,-0.20024999999999998,-0.30305,-0.8187500000000001],[0.0009192388155424891,0.0009192388155425283,0.0023334523779156243,0.002687005768508879,0.0029698484809835258,0.003676955262170035,0.0037476659402886814,0.004030508652763328,0.005161879502661778,0.006576093065034874],['g-575','g-23','c-73','g-675','c-29','g-189','g-257','c-19','g-402','g-422'])],[('72 * D2',[0.5224500000000001,-0.0031999999999999997,-0.68335,0.2721,0.14485,-0.34035,-0.27225,0.21565,-0.3388,0.1601],[0.0006363961030679012,0.0008485281374238571,0.0009192388155425676,0.0009899494936611753,0.0014849242404917434,0.0014849242404917434,0.0016263455967290372,0.0021920310216782916,0.0022627416997969383,0.0032526911934581135],['g-355','g-203','g-501','g-597','g-318','g-323','g-467','g-680','g-104','g-464']),('24 * D2',[-0.8269,0.36529999999999996,0.79945,-0.9792,-0.2296,-0.49295,0.08299999999999999,-0.0379,0.37245,-0.0695],[0.00028284271247458785,0.0009899494936611753,0.002474873734152958,0.0026870057685088986,0.0029698484809834867,0.0030405591591021333,0.005939696961967003,0.006505382386916237,0.0072831998462214225,0.007636753236814706],['g-698','g-35','g-643','g-124','g-127','c-41','g-455','g-392','g-1','g-618'])],[('24 * D2',[0.15394999999999998,0.49575,0.18045,0.7564,-0.65645,0.80345,-0.17909999999999998,-0.58245,-0.00915,-0.44265],[0.00021213203435596054,0.0003535533905932741,0.000494974746830568,0.0007071067811865482,0.001767766952966331,0.002333452377915664,0.002687005768508879,0.0028991378028648397,0.0031819805153394643,0.003889087296526015],['g-307','g-383','g-680','g-61','g-263','g-549','g-282','g-340','g-696','g-758']),('24 * D1',[0.65025,0.1335,0.5102,-0.027450000000000002,0.07955,-0.46085,0.16945,0.30810000000000004,-0.3853,0.59675],[7.071067811864696e-05,0.0007071067811865482,0.0015556349186103902,0.0016263455967290594,0.0017677669529663704,0.0021920310216782916,0.002333452377915605,0.004101219330881955,0.005798275605729718,0.006576093065034952],['g-186','g-672','g-509','g-334','g-634','g-340','g-136','g-103','g-201','g-625'])],[('48 * D1',[-0.19345,-0.42035,-0.0262,-0.1881,-0.36665000000000003,-0.49115,0.4308,0.138,0.32399999999999995,-0.39480000000000004],[0.0004949747468305876,0.0004949747468306073,0.0007071067811865482,0.00155563491861041,0.0019091883092036642,0.0019091883092036642,0.0035355339059327407,0.0036769552621700543,0.005232590180780464,0.005515432893255052],['g-266','c-96','g-276','g-495','g-411','g-373','g-26','g-512','g-314','g-736'])],[('48 * D1',[0.7769,0.3249,0.19569999999999999,-0.16970000000000002,-0.6211,-0.3468,-0.14835,0.23695,0.82725,-0.18980000000000002],[0.0012727922061358025,0.0014142135623730963,0.002262741699796958,0.0029698484809835063,0.0033941125496954466,0.003676955262170035,0.003889087296526015,0.0045961940777125634,0.0045961940777126025,0.004808326112068524],['g-52','g-381','g-358','g-528','g-629','g-161','g-545','g-288','c-96','g-269'])],[('48 * D2',[0.6698500000000001,0.43545,-0.017,-0.27365,-0.5211,0.1443,0.75785,-0.16895,-0.5474,0.66675],[0.00021213203435601938,0.0006363961030679012,0.0016970562748477136,0.0021920310216782916,0.0028284271247461927,0.0035355339059327407,0.004313351365237936,0.004313351365237936,0.0059396969619669735,0.007424621202458795],['g-694','g-747','g-343','g-401','g-191','g-382','g-116','c-71','g-296','g-439']),('72 * D1',[0.0955,-0.24480000000000002,-0.15465,0.53435,0.71865,0.1459,0.10535,0.22465,0.030799999999999998,-0.19190000000000002],[0.00014142135623731355,0.0008485281374238617,0.0009192388155425087,0.0009192388155425676,0.0013435028842544493,0.0022627416997969383,0.003464823227814084,0.003889087296526015,0.004242640687119284,0.0046669047558312296],['g-704','g-176','g-294','g-58','c-49','g-172','g-585','c-52','g-632','g-209'])],[('48 * D1',[0.33309999999999995,-0.09115000000000001,0.66595,-0.3328,0.00945,-1.2225000000000001,-0.9662,-0.9854,-0.52205,-0.5347500000000001],[0.0009899494936611753,0.001909188309203674,0.0023334523779155853,0.0025455844122715655,0.004454772721475249,0.004949747468305759,0.005091168824543131,0.005374011537017719,0.00601040764008562,0.006293250352560286],['g-204','g-63','g-490','g-237','g-453','c-75','g-250','g-761','g-749','g-163'])],[('48 * D1',[0.99685,-0.1839,0.3957,0.2457,0.31935,0.01455,0.1863,-0.7105,-0.3247,1.0514999999999999],[0.0009192388155425676,0.0009899494936611555,0.0011313708498985084,0.0015556349186103902,0.0017677669529663704,0.0020506096654409876,0.0021213203435596446,0.0031112698372207804,0.00466690475583121,0.004949747468305759],['c-25','g-183','c-18','g-556','c-86','g-424','c-22','g-683','g-648','g-554'])],[('24 * D1',[0.7093499999999999,-0.19795000000000001,-0.4683,-0.41464999999999996,0.3428,1.158,-0.46115,-0.2746,0.1867,-0.09104999999999999],[0.0007778174593051951,0.0012020815280171357,0.0015556349186103902,0.0019091883092036642,0.0021213203435596446,0.0028284271247461927,0.0037476659402886814,0.004101219330881995,0.005091168824543131,0.0054447222151364156],['g-151','g-68','g-392','g-621','g-50','g-630','g-502','g-674','g-212','g-681'])],[('48 * D1',[-0.1805,0.52,-0.6246499999999999,1.254,0.060399999999999995,-0.62425,-0.18680000000000002,-0.11845,-0.02835,0.74325],[0.0005656854249492346,0.0005656854249492542,0.0006363961030679012,0.0028284271247461927,0.0036769552621700495,0.004030508652763348,0.004384062043356602,0.005161879502661798,0.006576093065034894,0.00685893577750954],['c-22','g-722','g-602','g-436','g-254','g-448','g-158','c-75','c-28','c-67'])],[('24 * D1',[-0.31515000000000004,0.6113,-0.5581,0.1779,-0.3689,-1.9965000000000002,0.9661500000000001,-0.17365,-0.31010000000000004,0.08935],[0.00021213203435598016,0.0007071067811865482,0.000989949493661136,0.0021213203435596446,0.0025455844122715655,0.003535533905932819,0.005020458146424484,0.005444722215136405,0.005515432893255052,0.0072831998462214425],['c-20','g-749','g-334','g-59','g-427','g-639','g-477','g-500','g-188','g-22']),('48 * D1',[-0.11199999999999999,-0.5261,0.68715,0.26570000000000005,0.044649999999999995,-0.10345,0.34740000000000004,0.46304999999999996,-0.43069999999999997,0.0878],[0.00042426406871193083,0.0008485281374238421,0.0016263455967290372,0.003818376618407368,0.005020458146424489,0.005161879502661798,0.005374011537017758,0.0058689862838483655,0.006788225099390855,0.006929646455628168],['g-480','g-295','g-628','g-536','g-127','g-371','g-114','g-359','g-131','g-520'])],[('48 * D2',[0.015349999999999999,0.32785,0.6169,0.25315,0.38075000000000003,-0.0747,-0.05935,-0.4153,0.51325,0.4617],[0.00021213203435596422,0.0017677669529663704,0.0018384776310849781,0.0026162950903902125,0.003181980515339467,0.0035355339059327407,0.004030508652763319,0.0045254833995938765,0.005444722215136444,0.005656854249492385],['g-748','g-354','c-20','g-496','g-599','g-359','g-19','g-105','g-36','g-616'])],[('24 * D2',[0.44875,-0.53325,-0.6801999999999999,0.03975,1.0085,0.18105,-0.4122,-0.5854999999999999,0.059149999999999994,0.45509999999999995],[0.0009192388155425283,0.0009192388155425676,0.000989949493661136,0.0013435028842544395,0.002121320343559723,0.002192031021678311,0.0028284271247461927,0.0033941125496954466,0.004313351365237941,0.00466690475583121],['c-8','g-575','g-744','g-479','g-182','g-560','g-768','g-224','g-7','g-489'])],[('48 * D1',[0.0639,-0.18664999999999998,-0.02545,0.18230000000000002,0.34035000000000004,-0.21805000000000002,-0.11305,-0.35195,-0.30685,-0.3617],[0.00014142135623731355,0.00021213203435596054,0.00021213203435596541,0.0005656854249492346,0.0026162950903902125,0.002616295090390232,0.0037476659402887014,0.003747665940288721,0.004171930009000642,0.004242640687119289],['g-528','g-18','g-135','g-477','g-112','g-611','g-375','g-492','c-37','c-13']),('24 * D1',[-0.3513,0.63375,0.5434,0.2586,0.2904,0.14535,-0.96935,-0.3346,-0.28490000000000004,-0.19965],[0.0004242640687119603,0.0019091883092037035,0.0022627416997970167,0.002545584412271605,0.0026870057685088986,0.0028991378028648397,0.0028991378028648397,0.003676955262170035,0.004525483399593916,0.005161879502661798],['g-164','g-24','g-575','g-291','g-427','g-658','g-146','g-667','c-36','g-65']),('48 * D2',[-0.38385,0.43374999999999997,-0.77755,0.16975,0.3486,-0.35035,-0.5274,1.0006,-0.60685,-0.4591],[0.000494974746830568,0.0010606601717798223,0.0024748737341528794,0.002616295090390232,0.0029698484809835258,0.0030405591591021333,0.004101219330881995,0.004808326112068543,0.008555992052357225,0.00933380951166242],['c-52','g-269','g-21','g-669','g-674','g-187','g-424','g-293','g-77','g-248'])],[('24 * D2',[2.5595,0.9033,1.2505,1.532,-1.991,-0.45145,-0.8644499999999999,2.0875,-0.22165,-1.048],[0.0007071067811864697,0.002404163056034311,0.003535533905932662,0.004242640687119289,0.004242640687119289,0.00756604255869605,0.008273149339882558,0.009192388155425047,0.00954594154601838,0.009899494936611674],['g-746','g-321','g-480','g-144','g-691','g-65','g-329','g-413','g-424','c-50']),('48 * D2',[1.7595,-6.1635,-0.64915,-1.3599999999999999,-0.25625,5.13,-0.5508,-8.877500000000001,0.1305,4.037],[0.0007071067811866267,0.004949747468305602,0.005444722215136444,0.007071067811865481,0.007848885271170676,0.008485281374238892,0.009050966799187833,0.009192388155425047,0.009192388155425127,0.009899494936611204],['g-19','c-38','g-538','c-72','g-466','g-142','g-387','g-298','g-627','g-570']),('72 * D2',[2.316,-0.25665000000000004,0.6969000000000001,1.4515,-0.6661999999999999,-1.3415,-2.69,4.621,3.2714999999999996,-3.2969999999999997],[0.0014142135623732533,0.0019091883092036642,0.003111269837220859,0.003535533905932662,0.0035355339059327407,0.003535533905932819,0.004242640687119446,0.005656854249492385,0.006363961030678855,0.007071067811865638],['g-484','g-515','g-675','g-442','g-157','g-24','g-221','g-243','g-527','g-296']),('72 * D1',[-2.1425,-0.5161,-2.8475,0.5648500000000001,-1.2574999999999998,-4.0385,-0.1239,-2.0780000000000003,-3.802,-5.8015],[0.0007071067811864697,0.0026870057685088986,0.003535533905932662,0.006151828996322992,0.006363961030679012,0.006363961030679169,0.008485281374238558,0.008485281374238578,0.009899494936611831,0.012020815280170927],['g-273','g-459','c-73','g-132','g-730','g-379','g-734','g-479','g-175','c-6']),('24 * D1',[-0.6181,-0.5744,-1.3035,-3.7765,-5.7035,4.176,0.3394,-1.1389999999999998,1.0419999999999998,-1.3485],[0.002404163056034311,0.003959797974644701,0.006363961030679012,0.006363961030679169,0.006363961030679169,0.007071067811865324,0.009192388155425127,0.009899494936611674,0.012727922061357868,0.013435028842544494],['g-60','g-329','g-419','g-450','g-37','g-147','g-541','g-395','g-345','g-688'])],[('48 * D2',[-0.0204,-0.020499999999999997,0.6966,-0.37715,-0.12614999999999998,0.51425,0.49145,-0.02195,-0.36795,0.0668],[0.0001414213562373111,0.0005656854249492371,0.0008485281374239206,0.0016263455967290765,0.002333452377915605,0.0026162950903902515,0.0030405591591021333,0.006151828996322963,0.0071417784899841285,0.0076367532368147115],['g-503','g-560','g-133','g-310','g-0','g-225','g-304','g-472','g-270','g-685']),('24 * D2',[0.4215,-0.07295,0.2639,0.23854999999999998,0.6851499999999999,0.33335000000000004,0.3605,0.5701499999999999,0.29410000000000003,-0.78145],[0.00014142135623729392,0.0010606601717798223,0.0025455844122715655,0.003464823227814074,0.0036062445840513878,0.004171930009000603,0.004384062043356583,0.0045961940777126025,0.005091168824543171,0.006434671708797581],['g-216','g-369','g-639','c-3','g-667','g-597','g-353','g-487','g-502','g-510']),('24 * D1',[0.10005,0.12969999999999998,0.3368,-0.32065,-0.05555,0.25385,-0.35145,-0.26105,0.9317,0.7397],[0.00021213203435596054,0.0011313708498984889,0.0015556349186103902,0.004030508652763309,0.004596194077712558,0.004737615433949857,0.00487903679018719,0.005303300858899111,0.005656854249492385,0.0059396969619669735],['g-318','g-75','g-459','c-46','g-73','g-121','g-427','g-466','g-484','g-514']),('72 * D2',[-0.16815000000000002,-0.40395000000000003,-0.6712,-0.3388,0.1207,0.12129999999999999,0.28645,0.2754,0.0998,0.357],[0.00021213203435596054,0.0010606601717798223,0.0014142135623730963,0.0022627416997969383,0.0028284271247461927,0.0029698484809834963,0.005444722215136405,0.006081118318204307,0.006363961030678933,0.006505382386916227],['g-262','g-170','g-575','g-104','g-651','g-578','g-452','g-357','c-89','c-92'])],[('24 * D1',[-0.0954,-0.07105,-0.39035,0.1383,0.6062000000000001,0.1688,0.41835,-0.28700000000000003,-0.35375,0.0785],[0.0005656854249492346,0.0010606601717798223,0.0010606601717798223,0.0014142135623730963,0.003111269837220859,0.004949747468305837,0.008273149339882636,0.008485281374238538,0.008980256121069147,0.009475230867899732],['c-78','g-64','g-51','g-718','g-479','g-409','g-437','c-12','g-670','c-28'])],[('24 * D1',[-0.4376,0.5542499999999999,0.025099999999999997,-0.309,-0.7969999999999999,-0.13145,-0.40225,-0.44305,0.35385,-0.29569999999999996],[0.0007071067811865482,0.0009192388155424891,0.0012727922061357853,0.0035355339059327407,0.004666904755831249,0.005161879502661798,0.005444722215136444,0.006434671708797581,0.006717514421272168,0.007495331880577403],['g-494','g-709','g-224','g-194','c-45','g-728','g-102','g-152','g-420','g-766']),('24 * D2',[-0.29845,-0.26739999999999997,0.1136,-1.363,-1.0645,-0.00845,-0.20579999999999998,-0.31189999999999996,-0.33985,-0.2005],[7.071067811864696e-05,0.0011313708498984691,0.0016970562748477138,0.0028284271247461927,0.003535533905932662,0.00431335136523794,0.006505382386916247,0.006788225099390855,0.008273149339882598,0.011455129855222084],['g-270','g-766','g-593','c-25','c-1','g-550','g-470','g-602','g-637','g-249']),('48 * D2',[-0.26705,0.4335,0.30905,0.1456,-0.1851,0.08585,-0.7031000000000001,0.09035,0.1258,0.6349499999999999],[7.071067811864696e-05,0.0022627416997969383,0.0026162950903902515,0.0031112698372208003,0.004101219330881975,0.005020458146424484,0.005091168824543131,0.005161879502661798,0.006363961030678923,0.006434671708797581],['g-680','g-693','g-298','g-149','g-157','g-400','c-75','g-283','g-167','g-272'])],[('48 * D2',[-0.5102,0.22765000000000002,0.3422,-0.03555,-0.202,0.6532,-0.53125,-0.26455,0.6789499999999999,0.46504999999999996],[0.0015556349186103902,0.001909188309203684,0.0022627416997969383,0.0024748737341529184,0.002687005768508879,0.0031112698372207804,0.0044547727214752295,0.005444722215136444,0.006010407640085699,0.008697413408594559],['g-66','g-704','g-496','g-183','g-591','g-199','g-35','c-83','g-37','g-657']),('72 * D1',[-0.5307999999999999,0.42715000000000003,0.7594000000000001,-0.5949,-0.03615,-0.24595,-0.1768,-0.32030000000000003,-0.49665000000000004,-0.22585],[0.000989949493661136,0.002050609665440958,0.0021213203435596446,0.002404163056034311,0.0026162950903902272,0.0033234018715767605,0.004242640687119269,0.004384062043356622,0.0071417784899841675,0.00756604255869605],['g-31','c-11','g-673','g-259','g-610','g-247','c-28','g-302','g-248','g-332']),('48 * D1',[0.4175,0.5007999999999999,0.036449999999999996,-0.5617,0.3262,-0.64205,0.34914999999999996,-0.6315999999999999,0.62405,0.8551500000000001],[0.0002828427124746271,0.0007071067811865482,0.0010606601717798173,0.0014142135623730963,0.0015556349186103902,0.0034648232278140937,0.003889087296526015,0.005091168824543131,0.006434671708797581,0.006576093065034874],['c-59','c-37','g-609','g-282','g-423','g-235','g-97','g-709','g-450','c-13'])],[('48 * D1',[0.40195000000000003,0.5212,0.80935,-0.20024999999999998,0.0043,-0.17149999999999999,0.0463,-0.03185,-0.5213,0.12675],[0.00021213203435598016,0.00028284271247458785,0.0009192388155425676,0.0013435028842544298,0.0019798989873223336,0.0025455844122715854,0.0028284271247461927,0.0037476659402887035,0.004242640687119289,0.00487903679018718],['c-94','g-390','g-342','g-182','g-545','g-531','g-27','g-323','c-84','g-106'])],[('48 * D2',[0.8199000000000001,0.30884999999999996,-2.3360000000000003,-0.66815,1.529,0.27115,2.8425000000000002,3.7315,-1.282,1.584],[0.00028284271247466634,0.0017677669529663704,0.0028284271247461927,0.008980256121069107,0.01131370849898477,0.01817264427649427,0.02050609665441013,0.021920310216782757,0.02687005768508883,0.028284271247461926],['g-764','g-16','g-318','g-104','g-132','g-25','g-278','g-135','g-89','g-151']),('72 * D2',[-0.8080499999999999,4.3015,-1.0499999999999998,-8.8375,-8.8375,1.0554999999999999,-1.5910000000000002,-3.731,-1.919,0.4849],[0.0020506096654409976,0.002121320343559723,0.004242640687119289,0.012020815280170927,0.012020815280170927,0.012020815280171397,0.012727922061357868,0.014142135623730963,0.014142135623730963,0.020364675298172603],['g-188','g-478','g-562','c-26','c-41','g-166','g-89','g-206','g-296','g-99']),('24 * D2',[2.9684999999999997,-4.1850000000000005,0.32320000000000004,-8.117,1.7335,-1.962,2.2744999999999997,2.115,-1.506,-1.0579999999999998],[0.0007071067811864697,0.004242640687118818,0.006788225099390855,0.008485281374237636,0.009192388155425047,0.01131370849898477,0.013435028842544178,0.014142135623730963,0.016970562748477157,0.018384776310850254],['g-711','g-369','g-352','g-257','g-750','g-332','g-178','g-622','g-102','c-14']),('72 * D1',[-1.1524999999999999,2.1805,-6.121499999999999,0.11945,-0.5357,-0.97985,-7.447,-6.1690000000000005,3.601,-8.6555],[0.002121320343559566,0.002121320343559723,0.003535533905932662,0.005303300858899111,0.005656854249492385,0.006434671708797659,0.014142135623730649,0.014142135623731277,0.015556349186104216,0.016263455967290372],['g-371','g-408','g-360','g-403','g-184','g-200','c-81','g-178','g-124','c-40'])],[('48 * D1',[-0.3536,-0.0833,2.1035,1.4945,0.38980000000000004,0.21860000000000002,0.00039999999999999975,-1.784,-0.2519,1.6349999999999998],[0.00014142135623733317,0.0007071067811865482,0.002121320343559723,0.003535533905932819,0.003676955262170074,0.00622253967444162,0.00820243866176395,0.008485281374238578,0.008485281374238578,0.009899494936611674],['g-697','g-562','g-60','g-210','g-517','g-716','g-286','c-90','g-651','g-535'])],[('48 * D1',[0.11555,-0.5891,-0.45365,-0.6149,0.7081,-0.8933,0.63805,0.7456499999999999,-0.19335,0.4216],[0.00021213203435596054,0.0016970562748477628,0.0021920310216782916,0.002404163056034311,0.00268700576850882,0.0031112698372207804,0.004313351365237936,0.0047376154339498966,0.005444722215136405,0.005515432893255052],['g-433','g-437','g-210','g-683','g-680','g-180','g-331','c-70','g-203','c-51'])],[('48 * D1',[0.0668,0.38835,0.5716,0.27649999999999997,-0.36665000000000003,-0.40365,0.08645,0.58995,-0.1839,-0.34009999999999996],[0.0011313708498984691,0.0012020815280171162,0.0015556349186103902,0.0022627416997969777,0.003747665940288721,0.004313351365237936,0.004454772721475249,0.005727564927611032,0.006081118318204306,0.007495331880577403],['g-270','g-668','c-46','g-14','g-436','g-513','g-341','c-42','g-487','g-666'])],[('24 * D2',[0.7471,-0.554,0.37024999999999997,-0.7725,0.20879999999999999,-1.087,0.50135,1.0745,0.1376,-0.2447],[0.0012727922061358025,0.0021213203435596446,0.003181980515339467,0.0038183766184073284,0.004808326112068524,0.005656854249492385,0.006293250352560286,0.006363961030678855,0.01244507934888324,0.013293607486307082],['c-49','g-333','g-292','c-72','g-60','g-499','g-154','g-228','g-78','g-704'])],[('48 * D1',[-0.7296499999999999,0.84855,-0.13015,-0.71865,0.07915,0.09175,-0.19495,0.28765,-0.5205,0.30505000000000004],[0.0009192388155424891,0.0013435028842544493,0.0017677669529663704,0.0026162950903902515,0.0031819805153394665,0.003464823227814084,0.004879036790187171,0.005020458146424484,0.005232590180780425,0.005444722215136405],['g-174','c-86','g-649','g-321','g-141','c-20','g-373','g-356','g-6','g-526'])],[('48 * D1',[-0.05945,-0.16055,0.07705000000000001,0.90575,-1.565,-0.019700000000000002,0.7514000000000001,0.78995,0.8301000000000001,-0.46815],[7.071067811865187e-05,0.00021213203435598016,0.001202081528017126,0.0014849242404918216,0.0028284271247461927,0.0031112698372208077,0.0038183766184073284,0.004171930009000642,0.004384062043356661,0.0044547727214752295],['g-467','g-337','g-311','g-708','c-80','c-31','g-289','g-164','g-506','g-605'])],[('48 * D1',[0.2175,0.65745,-1.1515,-1.01,1.3345,-0.248,-0.4948,0.20105,0.71855,-1.2625],[0.0008485281374238421,0.0013435028842544493,0.002121320343559723,0.0028284271247461927,0.004949747468305916,0.006363961030678933,0.007636753236814696,0.007707463914933363,0.008131727983645342,0.009192388155425047],['g-21','c-86','g-720','g-135','c-43','g-517','g-628','g-575','g-231','g-374'])],[('24 * D1',[-0.6729499999999999,0.7618,-0.8765000000000001,-0.67475,-0.13835,-0.27264999999999995,-0.37005,0.2384,0.16044999999999998,0.4588],[0.00021213203435601938,0.0014142135623730963,0.0021213203435596446,0.002616295090390173,0.0028991378028648397,0.003181980515339467,0.0044547727214752295,0.005091168824543131,0.006151828996322973,0.006929646455628148],['g-239','g-271','g-562','g-94','g-584','g-46','g-497','g-268','g-427','g-211']),('72 * D1',[-0.24345,-0.38854999999999995,0.28215,-0.059399999999999994,-0.5613,0.66315,0.39065,-1.0074999999999998,-0.15705000000000002,-0.0069],[0.0007778174593051951,0.0010606601717798223,0.0028991378028648397,0.0032526911934581187,0.004101219330881995,0.0045961940777125235,0.00601040764008562,0.006363961030679012,0.009404520189781086,0.009758073580374355],['g-633','g-177','c-56','g-666','g-766','g-716','g-101','g-396','g-352','g-219'])],[('72 * D2',[-0.12915,-0.2891,0.83065,-0.3842,0.10535,-0.34885,-0.48575,0.21495,-1.0790000000000002,-0.8147],[0.0004949747468305876,0.0007071067811865482,0.0009192388155425676,0.0009899494936611753,0.0017677669529663704,0.0020506096654409976,0.003040559159102173,0.005727564927611032,0.007071067811865481,0.007495331880577363],['c-39','g-313','g-363','g-252','g-279','g-223','g-750','g-322','g-337','g-518']),('48 * D2',[-0.453,0.32465,0.48345,-0.04635,0.52355,0.047850000000000004,-0.1532,0.6302,0.3702,-0.7374499999999999],[0.0007071067811865482,0.0010606601717798223,0.0016263455967290372,0.0026162950903902272,0.004171930009000642,0.005161879502661793,0.005374011537017758,0.005515432893255091,0.007071067811865442,0.007848885271170676],['g-610','c-78','c-57','g-213','g-228','g-690','g-224','g-238','g-493','g-691']),('72 * D1',[0.67365,-0.37615,-0.1645,-1.1179999999999999,0.6308,0.1328,-0.52905,-0.0763,0.28745,0.83755],[0.00021213203435594088,0.000494974746830568,0.0011313708498984691,0.0014142135623730963,0.0026870057685088986,0.006505382386916227,0.006717514421272247,0.006788225099390864,0.006858935777509501,0.0071417784899841285],['g-366','g-662','g-12','g-758','g-369','g-221','g-208','g-354','g-108','g-178'])],[('48 * D1',[-0.08345,0.5558000000000001,-0.113,0.7297,-0.07819999999999999,0.36085,0.03245,0.007350000000000001,1.2879999999999998,-0.5808],[0.0003535533905932741,0.0004242640687119603,0.0012727922061357827,0.0022627416997969383,0.0028284271247461827,0.0030405591591021333,0.0031819805153394643,0.0033234018715767736,0.004242640687119289,0.004384062043356583],['g-430','g-192','c-96','g-267','c-3','c-41','c-38','c-54','g-300','g-723'])],[('48 * D1',[0.028650000000000002,0.28585,0.19805,-0.001,0.4968,-0.0258,-0.09145,0.83225,-0.3293,0.21994999999999998],[7.071067811865432e-05,0.0004949747468306073,0.0012020815280171357,0.001414213562373095,0.0014142135623730963,0.0014142135623730963,0.0027577164466275356,0.004313351365237936,0.004384062043356583,0.007707463914933363],['g-420','g-141','g-12','g-370','g-754','c-16','c-90','g-669','c-43','g-90'])],[('48 * D1',[0.32284999999999997,0.81425,-0.1634,0.29895000000000005,1.2105000000000001,0.6158,0.394,0.6163000000000001,0.5706,-0.059550000000000006],[0.0003535533905932741,0.0012020815280171554,0.0012727922061357827,0.0017677669529663704,0.002121320343559566,0.002404163056034311,0.0026870057685088595,0.005515432893255012,0.006505382386916306,0.009263098833543778],['g-574','g-296','g-161','c-84','g-325','g-109','g-656','g-604','g-537','c-86'])],[('48 * D2',[0.5457000000000001,-0.0265,0.32489999999999997,-0.35325,-0.2353,-0.5788,0.5934999999999999,-0.40959999999999996,0.10335,-0.94655],[0.0005656854249492542,0.0007071067811865482,0.0011313708498984691,0.0012020815280171554,0.002404163056034252,0.002404163056034311,0.0035355339059327407,0.004808326112068504,0.005020458146424484,0.0072831998462214225],['g-507','g-109','g-158','g-357','g-51','g-450','g-329','g-215','g-184','g-126']),('24 * D2',[-0.17104999999999998,-0.00015,0.0347,0.53545,0.11595,0.4551,-0.09240000000000001,0.051250000000000004,0.1902,0.6072],[0.00021213203435596054,0.00021213203435596425,0.0007071067811865482,0.0026162950903902515,0.003040559159102153,0.0031112698372207804,0.0032526911934581235,0.0038890872965260097,0.006363961030678933,0.007495331880577442],['g-705','g-307','g-636','g-305','g-501','g-225','g-401','g-661','g-460','g-320']),('72 * D1',[0.50275,-0.0212,-0.44035,-0.40035,0.29835,0.5064,-0.21534999999999999,0.22525,0.61415,0.83455],[7.071067811864696e-05,0.0012727922061357877,0.0014849242404917434,0.0019091883092036642,0.0019091883092037035,0.0022627416997969383,0.004171930009000622,0.004737615433949877,0.005727564927611032,0.008697413408594519],['g-79','g-326','c-44','c-18','g-497','g-279','g-89','g-442','g-540','g-306'])],[('48 * D1',[-0.2758,0.43679999999999997,0.8169500000000001,0.766,0.9372499999999999,0.19005,-0.3222,-0.53315,-0.33475,0.033100000000000004],[0.00014142135623729392,0.00042426406871192107,0.0023334523779155853,0.002404163056034311,0.0033234018715768,0.0036062445840513878,0.003676955262170035,0.004737615433949818,0.005161879502661778,0.006081118318204311],['c-58','g-435','c-5','g-753','g-228','g-373','g-351','g-15','c-69','g-598'])],[('48 * D1',[0.62205,-3.6545,-2.3375,-0.96865,-0.032549999999999996,0.7356,2.4735,0.1532,1.0040499999999999,-0.8583000000000001],[0.00021213203435594088,0.002121320343559723,0.002121320343559723,0.006576093065034952,0.007283199846221437,0.008768124086713167,0.009192388155425047,0.009616652224137047,0.011242997820866123,0.012303657992645907],['g-259','c-57','g-63','g-397','g-137','g-78','g-145','g-564','g-633','g-696'])],[('48 * D2',[0.09865,0.2892,0.438,0.65405,0.10214999999999999,0.38344999999999996,0.03375,-0.0063,-0.13105,0.16799999999999998],[0.00021213203435596054,0.0005656854249492542,0.0008485281374238421,0.0031819805153394274,0.004171930009000632,0.004737615433949857,0.004879036790187176,0.005798275605729689,0.006434671708797581,0.007636753236814716],['g-741','c-41','g-178','c-18','g-433','g-610','g-351','g-214','g-718','g-541']),('72 * D2',[0.55975,0.5386500000000001,0.24375,-0.37045,-1.097,0.0419,0.02345,0.67425,-0.2572,0.44715],[0.00021213203435594088,0.0006363961030679012,0.0021920310216782916,0.0026162950903902515,0.0028284271247461927,0.004242640687119289,0.004596194077712561,0.004737615433949818,0.004949747468305837,0.005586143571373738],['c-9','g-36','g-397','c-5','g-422','g-567','g-734','g-508','g-603','g-436'])],[('48 * D1',[0.47415,0.1643,0.91415,-0.001,-0.4364,0.19105,0.7893,-0.26915,0.4452,-0.08505],[7.071067811864696e-05,0.0002828427124746271,0.0013435028842544493,0.001414213562373095,0.0014142135623730963,0.0026162950903902125,0.003818376618407407,0.0038890872965259755,0.005798275605729679,0.006293250352560267],['c-30','c-65','g-640','g-370','g-732','c-13','g-287','g-758','g-350','g-541'])],[('48 * D1',[-0.3457,-0.43745,-0.71705,0.36065,-0.60985,1.4285,0.30075,1.2745,-2.1755,-1.4905],[0.0019798989873223505,0.0037476659402886814,0.004313351365237936,0.0044547727214752685,0.005868986283848326,0.006363961030679012,0.0071417784899841285,0.009192388155425047,0.009192388155425047,0.009192388155425205],['g-37','g-653','g-117','g-190','g-425','g-385','g-643','g-311','g-579','g-640'])],[('48 * D1',[0.78715,-2.6265,1.2425000000000002,-0.58605,1.263,-1.4675,2.17,-1.936,-2.019,-0.32384999999999997],[0.00021213203435594088,0.0007071067811864697,0.003535533905932819,0.0038890872965259755,0.004242640687119289,0.004949747468305916,0.005656854249492385,0.007071067811865481,0.008485281374238578,0.008980256121069147],['g-277','g-36','g-537','g-382','g-57','g-67','g-366','g-684','g-613','g-548'])],[('72 * D2',[1.0750000000000002,-0.1928,-0.32125000000000004,0.41364999999999996,-0.0375,-0.4083,-0.1159,0.031599999999999996,-1.6644999999999999,0.31825000000000003],[0.0014142135623730963,0.0025455844122715655,0.0026162950903902125,0.0026162950903902125,0.0028284271247461927,0.0035355339059327407,0.0036769552621700443,0.0038183766184073558,0.007778174593051951,0.010394469683442242],['g-515','g-60','g-221','g-97','c-51','g-320','c-17','g-353','g-210','g-440']),('48 * D1',[0.43025,-0.62705,0.35834999999999995,-1.2360000000000002,-0.3225,0.69755,-0.39505,-0.19915,0.9088499999999999,-0.26075000000000004],[0.000494974746830568,0.0007778174593052736,0.0010606601717798223,0.0014142135623730963,0.0019798989873223505,0.0027577164466275456,0.0030405591591021333,0.0072831998462214425,0.0072831998462215015,0.00756604255869605],['g-156','g-606','g-195','g-570','g-757','g-186','g-499','g-555','g-277','g-235']),('72 * D1',[-0.10135,0.90825,-0.9996499999999999,-0.00675,-0.34795,0.45255,-0.64015,-1.00265,-0.96925,0.7659],[0.0004949747468305778,0.002474873734152958,0.0033234018715768,0.003464823227814083,0.0045961940777125634,0.005727564927611032,0.006151828996322914,0.006151828996322914,0.008555992052357225,0.009050966799187833],['g-579','c-47','g-606','g-635','g-443','g-151','g-647','g-474','g-484','g-321']),('24 * D2',[-0.5670999999999999,0.5528,0.6954,-0.3336,0.37685,0.26670000000000005,0.07185,-0.90815,0.8706,0.3679],[0.00014142135623737242,0.0014142135623730963,0.0016970562748476843,0.0016970562748477233,0.0021920310216782916,0.006081118318204307,0.0072831998462214425,0.00799030662740797,0.009192388155425127,0.011172287142747477],['c-66','c-69','g-91','g-638','g-733','g-356','g-453','g-178','g-276','g-98']),('48 * D2',[0.49545,1.6960000000000002,0.99225,0.5019499999999999,-0.35950000000000004,0.96955,-1.113,-0.5799000000000001,-0.397,0.39244999999999997],[7.071067811864696e-05,0.0014142135623730963,0.0014849242404917434,0.0020506096654409976,0.00311126983722082,0.0031819805153394274,0.005656854249492385,0.006081118318204346,0.007353910524340109,0.007424621202458756],['g-247','g-628','c-36','g-503','g-763','g-379','g-196','g-311','g-275','g-589'])],[('24 * D2',[-0.28569999999999995,0.45120000000000005,0.21855000000000002,0.7394000000000001,0.67935,1.0899999999999999,0.011949999999999999,-0.5534,-0.3238,0.1112],[0.0002828427124746271,0.0002828427124746271,0.0004949747468305876,0.0007071067811865482,0.0010606601717798615,0.004242640687119289,0.005020458146424487,0.00509116882454321,0.0072124891681028145,0.008768124086713196],['g-165','g-406','g-730','g-413','g-217','g-54','g-141','c-59','c-93','g-188']),('48 * D2',[0.4254,-0.3066,-0.16160000000000002,-0.29164999999999996,-0.5541,-0.055749999999999994,-0.18714999999999998,0.1913,0.46275,0.5234],[0.00028284271247458785,0.0005656854249492149,0.0005656854249492346,0.0009192388155425283,0.000989949493661136,0.001909188309203679,0.002616295090390232,0.0028284271247461927,0.003889087296526015,0.006788225099390893],['c-81','g-737','g-708','g-580','g-96','g-244','g-107','g-371','c-69','g-591']),('72 * D2',[0.5035499999999999,0.00015,-0.20500000000000002,0.74065,0.32265,-0.2103,-0.5789,-0.24280000000000002,-0.00935,-0.1567],[7.071067811872547e-05,0.00021213203435596425,0.0009899494936611555,0.0027577164466275456,0.0033234018715768,0.004242640687119269,0.004384062043356583,0.00622253967444162,0.01025304832720494,0.013152186130069788],['c-10','g-153','g-582','g-540','c-4','g-658','g-257','g-482','c-71','g-667'])],[('48 * D1',[-0.8904000000000001,5.388999999999999,-1.1045,0.10125,-1.214,-1.0819999999999999,0.5657000000000001,-1.4755,-0.5106999999999999,0.09555],[0.0008485281374238421,0.0028284271247465066,0.003535533905932662,0.004171930009000632,0.004242640687119289,0.004242640687119289,0.004949747468305837,0.004949747468305916,0.007495331880577442,0.008131727983645294],['g-532','g-175','g-71','g-500','g-358','c-57','g-740','g-564','g-451','c-67']),('24 * D2',[-0.6596,-0.39115,1.7229999999999999,-1.3675000000000002,0.5487,0.6467,0.62395,-0.44515,-0.27695,-0.7186],[0.0019798989873223505,0.0021920310216782916,0.004242640687119289,0.004949747468305759,0.005091168824543131,0.006081118318204346,0.006434671708797581,0.0072831998462214225,0.008555992052357225,0.00975807358037438],['g-208','c-91','g-717','g-580','g-55','c-80','g-575','c-25','g-476','g-727']),('72 * D2',[-1.4995,0.8281000000000001,0.5003500000000001,-2.3765,0.09609999999999999,-0.49035,-0.26849999999999996,2.3165,-0.6628499999999999,0.4093],[0.0007071067811864697,0.0015556349186104687,0.0017677669529664096,0.003535533905932662,0.0036769552621700443,0.00487903679018719,0.005232590180780464,0.006363961030679169,0.006434671708797581,0.0072124891681027755],['g-498','g-292','g-719','c-5','g-118','g-27','g-279','g-600','g-488','g-584']),('48 * D2',[2.2984999999999998,-0.2682,0.17575000000000002,-0.3468,-0.38875000000000004,-0.27075,-0.5267,2.2285000000000004,-0.90825,0.0593],[0.003535533905932976,0.004384062043356583,0.004454772721475249,0.006081118318204307,0.006151828996322954,0.0071417784899841285,0.010465180361560928,0.0106066017177983,0.01110157646462883,0.01131370849898476],['g-128','g-220','g-295','g-157','g-355','g-727','g-710','g-374','g-92','g-195']),('72 * D1',[-0.39315,0.43395,0.59865,-1.8805,1.9900000000000002,2.2664999999999997,0.1721,1.9515,-0.9831000000000001,0.32015],[0.0006363961030679012,0.0023334523779155853,0.0028991378028648397,0.003535533905932819,0.004242640687119289,0.004949747468305916,0.005091168824543131,0.006363961030678855,0.006363961030678934,0.0072831998462214225],['g-571','g-185','g-474','g-120','g-368','g-144','g-49','g-20','g-581','g-619'])],[('48 * D1',[0.08675,-1.9385,0.27690000000000003,1.1915,-0.13195,-1.337,-2.2889999999999997,-0.6119,0.25529999999999997,0.39444999999999997],[0.0006363961030678914,0.0007071067811866267,0.005515432893255052,0.006363961030678855,0.007566042558696069,0.008485281374238578,0.009899494936611518,0.010182337649086262,0.01088944443027285,0.013222896808188436],['g-642','g-276','g-520','g-456','g-20','c-55','c-82','g-473','g-291','c-64'])],[('48 * D1',[-0.999,0.02685,0.29200000000000004,0.13935,-0.48614999999999997,0.08215,-0.3892,0.2746,0.088,-0.65805],[0.00028284271247458785,0.0006363961030678939,0.0011313708498984691,0.0013435028842544493,0.0017677669529663704,0.0031819805153394665,0.0033941125496954466,0.003676955262170035,0.003818376618407358,0.003889087296526054],['c-81','g-94','g-648','g-768','g-468','c-62','g-62','g-360','g-132','g-578'])],[('48 * D1',[0.26835,-0.2387,0.7980499999999999,-0.1822,-0.53795,0.8727,-0.41785,-1.459,0.5706,0.0606],[0.0007778174593051951,0.0009899494936611753,0.001060660171779783,0.0015556349186103902,0.0030405591591021333,0.003252691193458153,0.0036062445840513878,0.004242640687119289,0.004666904755831171,0.005656854249492385],['g-754','g-671','g-161','g-165','g-230','g-498','c-94','c-93','g-142','g-65'])],[('48 * D1',[0.28065,-0.0242,-0.1402,0.5603,0.6144499999999999,0.11660000000000001,0.14795,0.37434999999999996,0.48945,-0.128],[7.071067811864696e-05,0.0008485281374238568,0.001979898987322331,0.0022627416997969383,0.0023334523779155853,0.005091168824543141,0.005586143571373738,0.006858935777509501,0.00685893577750954,0.006929646455628158],['g-503','c-69','g-738','g-225','g-353','g-117','g-703','g-383','g-405','g-31']),('48 * D2',[-0.0617,0.15695,0.19874999999999998,0.5168999999999999,-0.19615,-0.34695,0.22145,-0.91255,0.5364500000000001,-0.1956],[0.0002828427124746173,0.0014849242404917434,0.002192031021678311,0.0022627416997969383,0.002333452377915605,0.0023334523779156243,0.003889087296526015,0.0045961940777125235,0.005020458146424484,0.005091168824543131],['g-342','g-119','g-344','c-95','g-146','c-71','g-553','g-51','g-586','g-446']),('72 * D1',[0.36385,0.00135,0.67255,-0.9161999999999999,-0.1094,-0.2848,0.06555,-0.8221499999999999,0.49675,-0.78755],[0.0016263455967290372,0.0019091883092036783,0.0019091883092037035,0.0021213203435596446,0.002687005768508879,0.0028284271247461927,0.0036062445840513878,0.003747665940288682,0.004737615433949857,0.0047376154339498966],['c-36','g-333','g-242','g-655','g-128','g-180','g-536','g-603','g-150','g-56']),('24 * D1',[0.28700000000000003,0.33325,0.13115,0.6335999999999999,-0.5234,-0.10300000000000001,0.3944,0.34850000000000003,-0.0185,0.1945],[0.0012727922061357632,0.0014849242404917434,0.0020506096654409776,0.003111269837220859,0.0032526911934580745,0.00466690475583122,0.005515432893255052,0.008343860018001244,0.008768124086713189,0.010323759005323596],['g-243','g-747','g-380','g-608','g-552','g-225','g-652','g-675','g-223','g-153'])],[('48 * D1',[0.4119,-0.4067,-0.58495,-0.7149000000000001,0.1162,-0.4556,0.06365,-0.3602,-0.1047,0.63005],[0.00014142135623729392,0.0016970562748476843,0.0028991378028648397,0.0033941125496954466,0.003959797974644662,0.005798275605729679,0.006576093065034894,0.007071067811865481,0.007353910524340089,0.009970205614730322],['g-126','g-441','g-293','g-333','g-576','g-361','g-234','g-497','g-465','g-77'])],[('72 * D1',[0.5305,0.12145,1.4500000000000002,-0.6124499999999999,-0.08505,-0.29315,-0.1679,-0.4393,0.1087,-0.13865],[0.0008485281374238421,0.0010606601717798223,0.0014142135623730963,0.0017677669529664096,0.007424621202458756,0.00756604255869605,0.009192388155425127,0.009899494936611635,0.01088944443027283,0.01110157646462879],['c-67','g-178','g-522','g-198','g-735','g-33','g-755','c-74','g-757','g-102']),('48 * D2',[-0.00015,0.6038,0.0337,0.0469,0.40190000000000003,-0.17645,1.181,0.11665,-0.00745,-0.0674],[0.00021213203435596425,0.00028284271247458785,0.0012727922061357827,0.004666904755831215,0.005515432893255052,0.0058689862838483455,0.008485281374238578,0.008838834764831842,0.010535891039679559,0.012020815280171303],['g-238','g-153','g-25','g-563','g-586','g-158','c-84','c-47','g-270','g-216']),('48 * D1',[0.05105,0.39165000000000005,0.05,-0.25665,1.1444999999999999,0.048100000000000004,0.33075,0.4072,-0.57925,0.4659],[0.00021213203435596541,0.0003535533905932741,0.00042426406871192595,0.0004949747468306073,0.0007071067811866267,0.0012727922061357877,0.0020506096654409976,0.004242640687119289,0.0047376154339498966,0.005091168824543131],['g-684','g-273','g-691','g-435','g-522','g-699','c-37','g-606','g-142','g-280']),('72 * D2',[-0.05255,-0.11515,-0.1111,0.58165,-0.89015,0.18335,0.5124,0.17185,0.6476999999999999,-0.172],[0.00021213203435596054,0.0012020815280171357,0.0018384776310850272,0.0021920310216783697,0.004313351365237936,0.004454772721475249,0.004949747468305837,0.005727564927611032,0.00622253967444164,0.0072124891681027955],['g-630','g-408','g-51','g-50','g-500','c-9','c-13','g-117','c-99','g-265'])],[('48 * D1',[0.024399999999999998,-0.45145,0.7556,-0.05775,0.3916,-0.72695,0.19745000000000001,0.5347,0.6638,0.02665],[0.0002828427124746198,0.0003535533905932741,0.000989949493661136,0.0010606601717798223,0.0012727922061358025,0.0013435028842544493,0.0014849242404917629,0.0022627416997969383,0.003959797974644622,0.004030508652763319],['g-764','g-75','g-590','g-165','g-459','g-382','g-269','g-511','g-760','g-483'])],[('24 * D1',[0.51515,-0.07844999999999999,0.25815,-1.1804999999999999,-1.0031999999999999,-1.167,-0.24905,1.1375,-0.2091,-0.29290000000000005],[0.00021213203435594088,0.0006363961030678914,0.0024748737341529184,0.004949747468305759,0.005374011537017719,0.005656854249492385,0.005727564927611032,0.006363961030678855,0.0072124891681027955,0.007495331880577403],['g-658','g-680','c-96','g-287','g-87','g-41','c-20','g-270','g-679','g-430'])],[('48 * D1',[-0.76205,0.33740000000000003,-0.40249999999999997,-0.12425,-0.18595,-1.687,0.8662,0.0009499999999999999,-0.1542,-0.00655],[7.071067811864696e-05,0.0002828427124746271,0.00042426406871192107,0.0006363961030679012,0.0020506096654409976,0.0028284271247461927,0.0029698484809834867,0.003464823227814083,0.007636753236814716,0.007848885271170678],['g-482','g-153','g-543','g-589','g-81','c-80','g-766','g-447','g-51','g-701']),('72 * D2',[-1.0825,-0.0798,0.237,0.58645,0.07805000000000001,-0.429,-0.4394,0.35750000000000004,-0.2702,0.5077],[0.0007071067811864697,0.0007071067811865482,0.0012727922061357827,0.0013435028842544493,0.003464823227814084,0.004949747468305837,0.005515432893255091,0.006081118318204307,0.007636753236814736,0.007636753236814736],['g-457','g-67','g-112','g-70','c-14','g-319','g-474','g-77','g-437','g-185'])],[('24 * D1',[0.62035,0.17070000000000002,0.7683,-0.18995,0.17830000000000001,-0.025,0.3188,0.5880000000000001,-0.21350000000000002,0.9097],[7.071067811864696e-05,0.0016970562748477038,0.0031112698372207804,0.00332340187157678,0.0046669047558312296,0.004949747468305834,0.0059396969619670125,0.006788225099390815,0.008061017305526637,0.00820243866176391],['c-73','g-243','g-304','g-713','g-533','g-649','c-82','g-364','g-229','c-91'])],[('48 * D1',[0.15175,-0.16799999999999998,1.3185,0.28600000000000003,0.41695000000000004,0.19445,-0.1253,0.37445,0.6103000000000001,0.53425],[0.00021213203435596054,0.00028284271247460747,0.002121320343559723,0.003818376618407368,0.003889087296526015,0.005020458146424504,0.005515432893255082,0.008555992052357225,0.008909545442950537,0.009121677477306401],['g-379','g-84','g-82','c-16','g-209','g-616','g-52','g-133','g-372','g-61'])],[('72 * D1',[-3.508,-1.9815,0.10930000000000001,-0.55305,-1.7475,3.3555,0.4704,-0.00014999999999999996,1.227,0.0548],[0.0014142135623729393,0.0106066017177983,0.01400071426749364,0.01421284630184961,0.014849242404917433,0.017677669529663625,0.019233304448274095,0.019586857838867367,0.019798989873223347,0.023900209204105304],['g-414','g-747','g-99','g-758','c-53','g-105','g-558','g-53','g-409','g-258']),('24 * D1',[-0.12225,-0.2279,3.4154999999999998,-0.44805,-0.96695,1.2045,0.9229,0.90655,0.469,0.1032],[0.00021213203435596054,0.0008485281374238617,0.002121320343559409,0.0033234018715768,0.004171930009000642,0.004949747468305759,0.005091168824543131,0.005161879502661778,0.005232590180780464,0.005798275605729699],['g-203','g-456','g-235','g-479','g-445','g-291','g-163','g-721','g-637','g-480']),('48 * D2',[-0.81985,-1.6425,-1.1975,1.2725,-0.2946,0.0028,-2.0555,-1.848,0.5327999999999999,0.25980000000000003],[0.0013435028842544493,0.002121320343559566,0.002121320343559723,0.002121320343559723,0.0022627416997969777,0.003959797974644666,0.007778174593052108,0.008485281374238578,0.010323759005323634,0.01088944443027285],['g-190','g-667','g-563','c-90','g-283','g-219','g-573','g-53','g-460','g-754'])],[('48 * D2',[0.86715,-0.22015,-0.49905,-0.2013,1.3119999999999998,0.72725,-0.1733,-0.33085,-0.33945000000000003,0.16435],[7.071067811864696e-05,0.0003535533905932741,0.002050609665440958,0.0025455844122715655,0.004242640687119289,0.006293250352560286,0.006505382386916247,0.007707463914933344,0.008273149339882598,0.009263098833543773],['g-13','g-85','g-728','g-216','g-624','g-655','g-294','c-55','g-686','g-296']),('72 * D2',[0.8707,0.5730999999999999,-0.08349999999999999,-0.15555,0.8579,-0.08149999999999999,0.50175,-0.39275,-0.44810000000000005,0.6210500000000001],[0.00014142135623729392,0.0011313708498984299,0.0025455844122715754,0.004313351365237936,0.0045254833995938765,0.0052325901807804545,0.0058689862838483655,0.006576093065034874,0.006788225099390855,0.007848885271170676],['c-0','g-364','g-632','g-252','g-246','c-68','g-612','g-259','g-244','g-268'])],[('48 * D1',[-0.57205,0.23895,-0.20784999999999998,-0.399,-0.26944999999999997,0.08045,0.1231,0.41600000000000004,0.72985,-0.0339],[0.0006363961030679012,0.0006363961030679012,0.001909188309203684,0.0024041630560342714,0.0026162950903902125,0.002616295090390222,0.0029698484809835063,0.0032526911934581135,0.003889087296526054,0.004525483399593903],['g-217','g-541','g-477','c-29','g-157','g-219','c-24','g-614','g-581','g-54']),('24 * D2',[-0.8953,0.32630000000000003,-1.3195000000000001,0.00185,-0.5662,0.31255,-0.43355,0.7790999999999999,-0.19385,-0.19290000000000002],[0.0009899494936612145,0.0011313708498984691,0.002121320343559566,0.002616295090390226,0.004101219330881995,0.004313351365237936,0.006293250352560286,0.006363961030678934,0.009687362902255693,0.00975807358037436],['g-191','g-605','c-71','g-104','g-624','g-586','g-674','c-97','g-438','g-547']),('72 * D1',[-0.00015,0.28505,-0.21484999999999999,-0.03005,-0.13435,0.2196,0.4853,0.22315000000000002,0.70135,0.11885],[0.00021213203435596425,0.000494974746830568,0.0004949747468305876,0.0007778174593052025,0.0021920310216782916,0.0032526911934581135,0.005656854249492385,0.006151828996322973,0.008273149339882636,0.00912167747730646],['g-370','c-41','g-334','g-81','g-588','g-517','g-249','c-14','g-605','c-51']),('24 * D1',[1.0525,1.0394999999999999,0.47555000000000003,-0.7624,-0.42385,-1.471,0.61825,0.45095,-0.49355000000000004,-0.2814],[0.0007071067811864697,0.002121320343559566,0.0023334523779156243,0.0026870057685088986,0.0034648232278140937,0.005656854249492385,0.006293250352560286,0.0071417784899841285,0.00756604255869605,0.008343860018001244],['g-98','g-302','g-68','g-757','g-252','g-508','g-504','g-75','g-663','g-31']),('72 * D2',[-0.1101,-0.20685,0.058499999999999996,0.69215,0.1776,-0.27444999999999997,-0.51695,-0.37695,-0.6549,-0.6646000000000001],[0.0007071067811865482,0.0014849242404917434,0.0016970562748477183,0.0017677669529664096,0.0018384776310850174,0.0023334523779156243,0.0027577164466275456,0.0034648232278140937,0.0038183766184073284,0.004949747468305837],['g-388','g-708','g-616','g-661','g-273','g-771','g-256','g-665','g-404','g-696'])],[('48 * D1',[0.020700000000000003,-0.78615,-0.033549999999999996,0.40115,-0.9501,1.855,-1.1324999999999998,-0.017750000000000002,0.26,0.40995],[0.00042426406871192844,0.0009192388155425676,0.0017677669529663656,0.0037476659402886814,0.003959797974644622,0.005656854249492385,0.006363961030679012,0.007141778489984131,0.009050966799187833,0.009970205614730322],['g-253','g-180','g-37','g-531','g-657','g-572','g-116','g-205','g-197','g-504'])],[('48 * D2',[0.11995,-0.20224999999999999,0.76095,-0.2639,-0.4175,-0.21225,-0.0387,0.0228,-0.3098,0.23149999999999998],[7.071067811864696e-05,0.0007778174593052147,0.003747665940288682,0.003959797974644662,0.005515432893255052,0.005586143571373719,0.005656854249492381,0.010323759005323592,0.010748023074035516,0.012020815280171298],['g-762','g-604','g-462','g-741','g-740','g-108','g-282','g-728','c-8','g-236']),('72 * D1',[-0.49929999999999997,-0.00415,0.43045,0.1058,0.17045,0.10155,0.6159,0.08355,0.5393,-0.01105],[0.00014142135623733317,0.0006363961030678926,0.0014849242404917434,0.002404163056034262,0.0031819805153394665,0.006576093065034894,0.006929646455628187,0.008273149339882598,0.009050966799187753,0.010394469683442249],['g-137','g-35','g-21','g-108','g-451','g-299','g-668','g-441','g-200','g-67']),('48 * D1',[-0.72195,0.4682,0.2918,-0.513,0.3182,1.057,0.53345,-0.4379,-0.0874,0.3467],[0.00021213203435594088,0.0008485281374238421,0.000989949493661136,0.0011313708498985084,0.0035355339059327407,0.004242640687119289,0.004313351365237936,0.004949747468305837,0.006646803743153551,0.006788225099390855],['g-699','g-597','g-134','g-668','g-28','g-79','g-670','g-460','g-418','g-595'])],[('48 * D2',[0.18745,-0.10745,-0.2683,0.35075,0.07139999999999999,-0.029699999999999997,-0.2814,0.5346500000000001,-0.1137,-0.5585],[7.071067811864696e-05,0.0003535533905932741,0.0025455844122715655,0.0028991378028648397,0.0029698484809834963,0.0038183766184073558,0.008061017305526656,0.008131727983645264,0.010323759005323596,0.010889444430272888],['c-18','c-74','g-291','g-77','g-8','g-244','g-516','g-558','g-459','g-599']),('48 * D1',[0.13515,0.4829,0.6855,-0.00095,1.109,0.2658,-0.054650000000000004,0.11199999999999999,-0.31325000000000003,-0.0945],[7.071067811864696e-05,0.0004242640687119603,0.0012727922061358025,0.0013435028842544404,0.0014142135623730963,0.0021213203435596446,0.00544472221513642,0.006505382386916237,0.008414570696119892,0.008768124086713186],['g-455','g-676','g-340','g-535','g-37','g-89','g-741','g-397','g-466','g-677']),('72 * D1',[0.6063000000000001,-0.30345,0.878,-0.26095,-0.15560000000000002,0.7562,-0.99535,-0.27955,0.3526,0.49219999999999997],[0.00014142135623737242,0.00021213203435594088,0.0009899494936612145,0.0021920310216782916,0.002262741699796958,0.0026870057685088986,0.003747665940288682,0.004313351365237936,0.004525483399593916,0.005091168824543171],['c-15','g-116','g-196','g-475','g-68','c-0','g-86','g-53','c-31','g-738']),('24 * D1',[-0.5897,0.43695,0.26065,-0.13615,0.1032,0.2868,0.18605,-0.2669,-0.65795,0.6972499999999999],[0.0005656854249491757,0.0007778174593051951,0.0023334523779155853,0.002616295090390232,0.005091168824543151,0.006081118318204346,0.006151828996322973,0.00664680374315356,0.0071417784899841285,0.007424621202458795],['g-92','c-17','g-54','c-51','g-457','g-378','g-388','c-34','g-602','g-111'])],[('48 * D1',[-0.3166,0.4516,-0.7475499999999999,0.09434999999999999,0.035449999999999995,0.10935,-0.033350000000000005,-0.5839,0.23165,-0.6811499999999999],[0.00042426406871192107,0.0015556349186103902,0.0016263455967291157,0.002192031021678301,0.0024748737341529136,0.002616295090390222,0.0028991378028648445,0.0031112698372207804,0.0031819805153394665,0.0034648232278140937],['g-169','g-633','g-218','g-29','g-481','g-399','c-44','g-546','g-739','g-480'])],[('48 * D1',[0.43585,-0.7314,-0.25360000000000005,0.3372,-0.66715,0.2504,-1.417,-0.27005,-0.6874,-0.87645],[7.071067811864696e-05,0.001272792206135724,0.0024041630560342714,0.0029698484809834867,0.004171930009000642,0.005374011537017739,0.005656854249492385,0.006151828996322954,0.006929646455628187,0.007707463914933382],['g-259','g-149','g-56','c-62','g-95','g-135','c-71','g-473','g-439','g-275'])],[('48 * D1',[0.2605,-1.7850000000000001,-2.6945,0.5727,2.1115,-1.3435000000000001,0.6702,0.048100000000000004,-2.4725,-0.31625000000000003],[0.0007071067811865482,0.0014142135623730963,0.002121320343559723,0.0022627416997969383,0.003535533905932662,0.003535533905932819,0.003818376618407407,0.005798275605729694,0.006363961030678855,0.006576093065034913],['g-31','g-7','g-329','g-485','g-678','g-357','g-42','g-747','g-62','g-209'])],[('48 * D1',[0.7864500000000001,0.06025,-0.001,-0.6173,-1.0965,-0.07680000000000001,0.20455,-0.1744,-0.5688,0.294],[0.0009192388155424891,0.0010606601717798223,0.001414213562373095,0.0021213203435596446,0.002121320343559723,0.00311126983722081,0.005444722215136425,0.007071067811865481,0.00735391052434007,0.008485281374238578],['g-387','g-499','g-370','g-393','g-109','g-372','g-698','g-322','g-56','g-167'])],[('48 * D2',[0.0065,0.83305,1.092,-1.027,0.38345,0.9533499999999999,-0.393,-0.3074,1.3199999999999998,0.48245000000000005],[0.0009899494936611668,0.0014849242404917434,0.0028284271247461927,0.0028284271247461927,0.0028991378028648397,0.005303300858899072,0.012020815280171319,0.012869343417595162,0.01555634918610406,0.015909902576697336],['g-384','g-101','g-371','g-168','g-618','g-391','g-308','g-199','g-161','g-375']),('72 * D2',[1.0775000000000001,-0.22705,0.2573,0.26539999999999997,-0.48815,0.01885,-0.27705,-0.5941000000000001,-0.2676,0.2556],[0.0007071067811866267,0.0007778174593051951,0.0014142135623730963,0.0026870057685088595,0.002757716446627506,0.0030405591591021555,0.0045961940777125634,0.004808326112068543,0.005232590180780425,0.005232590180780425],['c-1','g-286','c-23','g-280','g-20','g-396','g-635','g-183','g-542','g-291']),('24 * D1',[-0.18355,0.0125,-0.16895,0.17735,0.2646,-0.67785,0.54905,0.6268,-0.2471,0.27525],[0.0003535533905932741,0.0011313708498984752,0.0014849242404917629,0.001909188309203684,0.0019798989873223505,0.0028991378028648397,0.0030405591591021333,0.0033941125496954466,0.003818376618407348,0.004313351365237936],['g-263','g-371','g-297','g-95','g-57','g-426','g-257','g-21','g-115','g-5'])],[('48 * D1',[-0.83405,-0.62825,-0.7219500000000001,0.12780000000000002,-3.142,1.5725,-0.4232,0.61395,-0.14800000000000002,0.06835],[0.001060660171779783,0.002192031021678291,0.0033234018715768,0.0033941125496954275,0.004242640687119446,0.004949747468305916,0.005374011537017758,0.005586143571373738,0.006929646455628168,0.007566042558696059],['g-595','g-286','g-20','g-152','g-202','g-479','g-382','g-637','g-32','g-574'])],[('24 * D2',[-0.00015,0.00015,0.26805,0.02595,0.6205,-0.00075,-0.8512,0.12935,1.1705,0.15234999999999999],[0.00021213203435596425,0.00021213203435596425,0.0004949747468306073,0.001767766952966368,0.0022627416997969383,0.004171930009000631,0.006788225099390815,0.0071417784899841285,0.007778174593051951,0.009828784258493007],['g-164','g-307','g-571','g-17','g-595','g-126','g-217','g-241','g-16','g-678'])],[('48 * D1',[-0.8526,-0.27315,-0.0383,-0.2137,-1.0725,0.7152499999999999,-0.43410000000000004,-0.10625,-0.84665,0.4262],[0.00014142135623729392,0.0009192388155424891,0.0035355339059327407,0.004242640687119289,0.004949747468305916,0.005444722215136366,0.0059396969619670125,0.006576093065034894,0.006717514421272247,0.009333809511662459],['g-320','g-123','g-94','g-735','g-63','g-673','g-535','g-36','g-552','g-616']),('48 * D2',[-0.1645,0.95565,0.38815,0.39749999999999996,0.13025,-0.27215,1.353,-0.7605,-0.7333000000000001,0.14535],[0.00042426406871192107,0.0010606601717798615,0.0036062445840513878,0.003959797974644662,0.004171930009000642,0.005303300858899072,0.005656854249492385,0.0072124891681027755,0.009050966799187833,0.010960155108391496],['g-560','g-567','g-341','g-239','g-326','g-279','c-31','g-352','g-181','g-19']),('72 * D2',[-0.18445,-0.33075,0.19795000000000001,-0.041499999999999995,-0.5034,-0.9935499999999999,0.69875,-0.09154999999999999,-0.19705,0.8295],[0.0003535533905932741,0.0007778174593051951,0.0013435028842544298,0.002828427124746188,0.0029698484809834867,0.0033234018715768,0.0034648232278140937,0.0036062445840513977,0.005020458146424484,0.005656854249492385],['g-639','g-14','g-628','g-117','g-211','g-13','g-629','g-569','g-328','g-366'])],[('24 * D2',[0.00015,0.28364999999999996,0.64225,1.3555000000000001,0.8426499999999999,0.3772,0.43665,0.6743,0.4908,-0.7001],[0.00021213203435596425,0.0016263455967290765,0.002474873734152958,0.003535533905932819,0.0036062445840513878,0.008343860018001244,0.009970205614730322,0.011596551211459359,0.012727922061357866,0.013859292911256297],['g-23','c-25','g-704','g-193','g-679','g-110','g-431','g-590','g-154','g-46']),('48 * D2',[0.46695,0.3352,0.49235,0.41469999999999996,0.38605,-0.4445,-0.2571,0.2311,-0.09695000000000001,-0.0019],[7.071067811868621e-05,0.00014142135623729392,0.0014849242404917434,0.0024041630560342714,0.0030405591591021333,0.0031112698372207804,0.0045254833995938765,0.004949747468305837,0.005020458146424494,0.005232590180780452],['g-715','c-5','c-67','c-70','g-731','g-512','g-99','g-598','g-487','g-664'])],[('48 * D1',[0.49705,0.5079499999999999,0.1009,0.2601,-0.13984999999999997,0.09955,0.30920000000000003,0.08829999999999999,-0.13845000000000002,0.30765],[0.0003535533905932741,0.0007778174593051951,0.0012727922061357827,0.0012727922061358025,0.002333452377915605,0.002616295090390232,0.004525483399593916,0.006363961030678923,0.006858935777509521,0.007707463914933344],['g-419','c-50','g-163','g-561','g-677','g-681','g-91','c-92','g-261','g-506']),('72 * D1',[-0.03885,0.060200000000000004,0.5131,-0.3467,0.08025,0.34109999999999996,-0.3534,-0.5323,0.138,0.41795000000000004],[0.0010606601717798223,0.0012727922061357877,0.002121320343559566,0.0022627416997969383,0.0028991378028648397,0.00311126983722082,0.006363961030678933,0.006505382386916306,0.006788225099390854,0.007424621202458756],['g-58','g-596','g-688','g-273','g-114','g-679','g-1','g-479','c-19','g-50']),('24 * D1',[-0.59575,-0.1878,-0.4615,0.3264,0.011,-0.26954999999999996,0.003,0.1395,0.19295,0.0158],[0.00021213203435594088,0.00042426406871192107,0.0007071067811865482,0.0012727922061357632,0.0021213203435596433,0.003889087296526015,0.004242640687119285,0.0057982756057296985,0.0071417784899841285,0.010323759005323592],['g-259','g-411','g-405','g-368','g-340','g-5','g-747','g-212','g-124','g-148']),('72 * D2',[1.154,-0.47425,0.16115000000000002,-0.024550000000000002,0.63915,0.42364999999999997,0.25715,-0.26865,-0.24225000000000002,0.19095],[0.0014142135623730963,0.0016263455967290372,0.0020506096654409776,0.0026162950903902246,0.0030405591591021333,0.0033234018715767605,0.005727564927611032,0.006434671708797581,0.006576093065034894,0.0072831998462214425],['c-74','g-475','g-273','g-694','g-522','g-56','c-19','g-54','g-620','g-334'])],[('48 * D1',[0.69615,0.2082,0.40535,-0.09175,0.93375,0.8946000000000001,0.9837,-0.9794,0.3559,0.6039],[0.00021213203435601938,0.0007071067811865482,0.0009192388155425283,0.0012020815280171357,0.0014849242404917434,0.0015556349186104687,0.0021213203435596446,0.0022627416997969383,0.0025455844122715655,0.0028284271247461927],['g-369','g-31','g-569','g-401','g-300','c-2','c-93','g-443','c-18','c-5'])],[('72 * D1',[0.51825,-0.3593,0.4197,-1.1075,-0.42715000000000003,-0.2051,-0.18105,-0.47809999999999997,-0.46405,0.8307],[0.0004949747468305288,0.0005656854249492542,0.0007071067811865482,0.0007071067811866267,0.0012020815280171162,0.0016970562748477233,0.002616295090390232,0.003676955262170074,0.004171930009000642,0.004949747468305837],['g-71','g-471','g-303','c-20','g-22','g-516','g-33','g-404','g-60','g-388']),('48 * D1',[-0.00015,0.13074999999999998,-0.44775,-0.27815,-0.98375,0.8553999999999999,-0.3393,0.53485,-0.62825,0.0313],[0.00021213203435596425,0.0009192388155425087,0.0009192388155425283,0.0016263455967290372,0.001767766952966331,0.0019798989873223505,0.003676955262170035,0.003747665940288682,0.004313351365237936,0.0043840620433565946],['g-370','g-171','g-469','g-744','g-653','g-720','c-68','g-763','g-232','g-154'])],[('48 * D1',[0.53355,0.61925,0.11349999999999999,0.85065,-0.51485,-0.00235,0.9089499999999999,-0.9079999999999999,0.71705,0.057999999999999996],[0.0014849242404917434,0.0016263455967290372,0.0016970562748477138,0.001767766952966331,0.002192031021678291,0.0033234018715767736,0.005020458146424484,0.005232590180780425,0.005444722215136444,0.005656854249492381],['g-243','g-569','g-196','g-49','c-20','g-104','g-201','g-353','g-280','g-5'])],[('48 * D1',[0.63815,-0.7949,-1.3275000000000001,-0.72325,0.6645,-0.25405,-0.2378,-0.6931499999999999,-0.27125,-1.6640000000000001],[0.0006363961030678226,0.0014142135623730963,0.002121320343559566,0.002757716446627467,0.004384062043356583,0.0047376154339498966,0.004949747468305818,0.006717514421272168,0.007000357133746834,0.007071067811865481],['g-760','g-510','c-26','c-36','g-246','c-46','c-97','g-456','c-14','g-384'])],[('48 * D1',[0.00015,-0.79565,-0.2303,0.5178499999999999,-0.10569999999999999,0.1737,-0.51785,-0.28164999999999996,-0.2974,0.2573],[0.00021213203435596425,0.00035355339059331333,0.0021213203435596446,0.0036062445840513878,0.0036769552621700443,0.003959797974644662,0.0044547727214752295,0.005444722215136405,0.00622253967444164,0.007353910524340109],['g-370','g-612','g-548','g-701','g-113','g-257','g-164','c-3','g-29','g-408']),('24 * D2',[0.87715,0.5778000000000001,-0.0668,-0.025500000000000002,-0.3216,0.077,-0.3915,0.628,0.6051,0.68115],[0.00035355339059323483,0.0005656854249492542,0.0007071067811865482,0.001697056274847716,0.0038183766184073284,0.004242640687119289,0.004949747468305837,0.005798275605729679,0.00735391052434007,0.008697413408594519],['c-61','g-597','g-701','g-226','g-710','g-41','g-441','g-610','c-29','g-121'])],[('72 * D2',[0.45235000000000003,0.74045,0.06335,0.1732,0.7792,-0.06835,0.13095,-0.5183,1.317,-0.28125],[0.000494974746830568,0.0009192388155424891,0.0021920310216782964,0.0025455844122715655,0.0026870057685088986,0.0027577164466275356,0.004737615433949877,0.004808326112068543,0.005656854249492385,0.005727564927611032],['c-89','c-48','g-361','g-415','g-321','g-330','g-90','g-596','g-114','g-135']),('48 * D2',[-0.0013,0.03255,0.50185,0.0060999999999999995,0.3653,-0.01465,-0.38839999999999997,0.7203999999999999,0.22085,0.18285],[0.00042426406871192855,0.0010606601717798223,0.001767766952966331,0.0018384776310850241,0.0019798989873223505,0.0028991378028648445,0.0032526911934581135,0.0033941125496954466,0.006293250352560266,0.006576093065034894],['g-50','g-748','g-397','g-414','g-509','g-608','g-519','g-480','g-530','g-714']),('48 * D1',[-0.07919999999999999,0.0353,-0.93235,-0.22435,0.37095,0.7411000000000001,0.21195,-0.35375,0.2109,-0.97305],[0.0008485281374238519,0.0009899494936611655,0.0016263455967290372,0.0028991378028648397,0.0033234018715768,0.004949747468305837,0.005020458146424484,0.005161879502661778,0.00664680374315356,0.006717514421272168],['g-662','g-500','g-91','c-63','g-568','g-135','g-727','g-394','g-70','g-163'])],[('48 * D1',[-0.1116,0.2151,0.57035,-1.182,0.19785,0.32494999999999996,-0.687,1.2240000000000002,0.247,0.16199999999999998],[0.0002828427124746173,0.0008485281374238617,0.0013435028842544493,0.0014142135623730963,0.0021920310216782916,0.0026162950903902125,0.0029698484809835652,0.004242640687119289,0.006646803743153541,0.007353910524340089],['c-8','g-500','g-401','c-17','g-579','g-671','g-753','g-21','c-28','g-568'])],[('48 * D1',[0.1701,-0.11095,0.3542,-0.1094,-0.7646999999999999,-0.2513,-0.38705,0.18015,0.0429,-0.6365000000000001],[0.0002828427124746271,0.0004949747468305778,0.0008485281374238421,0.0028284271247461927,0.003252691193458153,0.0035355339059327607,0.0036062445840513878,0.004030508652763309,0.004242640687119289,0.004525483399593955],['g-367','g-756','g-178','c-74','g-277','g-275','c-88','g-682','g-679','g-90'])],[('72 * D2',[0.38465,-0.056999999999999995,-0.17604999999999998,-0.26205,0.3437,0.06895,0.44165,0.03745,-0.05075,-0.5962000000000001],[0.00021213203435594088,0.0005656854249492395,0.0006363961030678816,0.0016263455967290372,0.0025455844122715655,0.005020458146424484,0.005727564927611032,0.005727564927611037,0.008273149339882603,0.009192388155425127],['g-360','g-722','g-575','g-492','g-513','g-37','g-304','g-186','g-41','g-277']),('24 * D1',[0.0538,0.221,0.56175,0.29495,-0.0674,0.39754999999999996,0.3994,-0.4296,-0.372,0.8469],[0.0002828427124746173,0.0011313708498984691,0.0013435028842544493,0.0020506096654409976,0.0021213203435596446,0.003040559159102173,0.004242640687119289,0.006222539674441601,0.006788225099390855,0.008485281374238578],['g-162','g-133','g-709','c-80','g-229','g-534','g-489','g-378','g-711','g-185'])],[('24 * D1',[-2.084,3.682,0.3937,0.3713,-1.8545,-1.3929999999999998,-2.1075,1.2075,2.0465,-1.8925],[0.0014142135623729393,0.0014142135623729393,0.0021213203435596446,0.004242640687119289,0.004949747468305916,0.007071067811865481,0.009192388155425047,0.01202081528017124,0.012020815280171555,0.013435028842544336],['g-253','g-437','g-27','g-111','g-354','g-53','g-725','g-707','g-683','g-301']),('72 * D2',[-2.8935,4.787,-9.9225,-0.7196,-0.8924,-0.94065,1.867,4.746,0.77015,-2.2104999999999997],[0.0007071067811867837,0.0028284271247458787,0.007778174593052736,0.008485281374238578,0.010040916292848968,0.010818733752154164,0.01131370849898477,0.01272792206135771,0.013081475451951102,0.016263455967290685],['g-33','g-337','c-83','g-643','g-81','g-550','g-413','g-229','g-769','g-52']),('24 * D2',[0.9186,-2.3395,-2.901,-1.5305,1.1065,9.995000000000001,0.2519,-2.7270000000000003,-2.4345,-2.4355],[0.00014142135623729392,0.002121320343559723,0.004242640687119132,0.004949747468305916,0.006363961030678855,0.007071067811865324,0.007495331880577383,0.008485281374238578,0.01202081528017124,0.014849242404917747],['g-101','c-35','c-34','g-701','g-296','g-215','g-42','g-511','g-577','c-79']),('48 * D1',[4.5489999999999995,0.9582999999999999,3.7525,1.752,-3.9555,0.12489999999999998,2.3869999999999996,0.2066,-1.258,5.269],[0.0014142135623729393,0.0016970562748477628,0.002121320343559723,0.0028284271247461927,0.006363961030678855,0.006363961030678923,0.008485281374238578,0.009192388155425127,0.009899494936611674,0.014142135623730649],['g-427','g-376','g-442','g-658','g-167','g-239','g-630','g-227','g-614','g-589']),('48 * D2',[0.69495,1.303,8.164,-2.0374999999999996,-9.996,-8.7775,-1.3864999999999998,-0.97575,0.07369999999999999,-4.508],[0.0024748737341528794,0.004242640687119289,0.004242640687119446,0.004949747468305916,0.005656854249491757,0.006363961030679169,0.007778174593051951,0.008555992052357225,0.011313708498984762,0.01131370849898477],['g-631','g-306','g-724','g-238','g-50','c-70','c-74','c-61','g-430','g-450']),('72 * D1',[5.8035,-5.893,0.03145,1.7955,-1.0305,-1.5805,0.19924999999999998,-3.3949999999999996,-0.19155,0.85815],[0.003535533905932662,0.008485281374238892,0.008980256121069152,0.010606601717798144,0.0106066017177983,0.014849242404917433,0.015909902576697315,0.019798989873223347,0.020576807332528545,0.02170817818242705],['g-373','c-35','g-763','g-233','g-305','g-652','g-493','g-160','g-383','g-718'])],[('48 * D1',[0.6423,0.1432,0.8664499999999999,-0.133,-0.48435,0.0005499999999999999,-1.101,0.60465,0.19735,-0.1918],[0.00028284271247458785,0.0002828427124746271,0.0004949747468306073,0.0011313708498984691,0.0014849242404917434,0.0021920310216782973,0.0028284271247461927,0.0030405591591021333,0.0031819805153394665,0.004525483399593916],['g-431','g-532','g-256','g-228','g-144','g-652','g-84','g-315','g-517','g-686'])],[('48 * D1',[-0.6627000000000001,3.1975,-1.5065,0.4879,-0.77555,-1.737,1.5885,-1.9415,1.1345,-0.2518],[0.0016970562748477628,0.002121320343559409,0.003535533905932662,0.005515432893255091,0.006293250352560286,0.007071067811865481,0.007778174593052108,0.009192388155425047,0.012020815280171397,0.013293607486307082],['g-413','g-108','g-704','g-620','g-315','g-485','g-685','g-701','g-177','g-181'])],[('48 * D1',[0.94255,-0.1403,-0.26545,0.3608,0.55385,0.0238,0.1073,-0.203,0.28164999999999996,0.3269],[7.071067811864696e-05,0.00042426406871192107,0.0021920310216782916,0.0024041630560342714,0.0028991378028648397,0.002969848480983499,0.00311126983722081,0.0033941125496954275,0.003889087296526015,0.004242640687119289],['g-466','c-77','c-16','g-409','g-735','g-763','c-89','c-96','g-145','c-46'])],[('48 * D1',[0.29305000000000003,-0.5383,0.41125,0.68415,0.0563,-0.4235,-0.12295,0.7758499999999999,-0.2647,-0.6104499999999999],[0.00021213203435598016,0.00042426406871188177,0.0004949747468306073,0.0013435028842544493,0.0014142135623730913,0.0014142135623730963,0.0020506096654409876,0.0023334523779155853,0.0028284271247461927,0.0030405591591022123],['g-748','g-296','g-61','c-10','g-726','g-228','g-320','c-11','g-625','g-75'])],[('48 * D1',[-0.49205,-0.4041,1.2995,-1.244,0.6441,-2.4665,-0.43889999999999996,-3.5875,-1.9385,1.1444999999999999],[0.0003535533905932741,0.0018384776310850174,0.002121320343559566,0.0028284271247461927,0.004808326112068465,0.004949747468305916,0.0059396969619670125,0.009192388155425047,0.009192388155425205,0.009192388155425205],['g-107','c-56','g-274','g-720','g-16','c-64','g-125','g-75','g-511','g-623'])],[('72 * D2',[0.133,0.68795,0.45389999999999997,0.1329,0.7464,-0.0224,-0.238,0.38545,0.2491,0.13640000000000002],[0.00042426406871192107,0.0014849242404917434,0.0016970562748477233,0.0021213203435596446,0.0025455844122715264,0.00311126983722081,0.0032526911934581335,0.004313351365237936,0.005091168824543131,0.005091168824543151],['g-160','g-584','g-635','g-100','g-60','g-637','g-112','c-23','g-693','g-534']),('24 * D2',[0.34955,0.04495,-0.4698,-0.41825,0.1016,-0.50445,-0.41435,0.10035,0.0582,-0.6738500000000001],[0.00021213203435598016,0.0003535533905932741,0.0012727922061358025,0.0026162950903902125,0.002687005768508879,0.0027577164466275456,0.004030508652763348,0.009263098833543773,0.010182337649086286,0.010960155108391456],['g-134','g-118','g-740','g-124','g-129','c-70','g-321','g-667','g-216','g-158'])],[('48 * D1',[-0.1461,-0.49834999999999996,0.4184,1.8205,0.28900000000000003,0.6255,-2.1005000000000003,0.41025,-0.8126,0.049449999999999994],[0.00155563491861041,0.0026162950903902125,0.0033941125496954466,0.003535533905932662,0.004384062043356622,0.004525483399593955,0.004949747468305916,0.00601040764008562,0.006505382386916227,0.00700035713374682],['g-540','c-98','g-90','g-33','g-434','g-196','c-81','g-654','g-214','g-761'])],[('24 * D1',[0.00435,-0.12755,-0.41275,0.21250000000000002,1.1055000000000001,0.40495000000000003,-0.6408499999999999,1.0425,0.4582,0.1763],[0.0003535533905932735,0.0006363961030679012,0.0009192388155424891,0.001979898987322331,0.003535533905932819,0.0045961940777125634,0.00487903679018719,0.004949747468305916,0.005656854249492385,0.0059396969619669926],['g-353','g-489','g-244','c-14','g-318','g-186','g-405','g-586','g-98','g-116']),('72 * D1',[0.14015,-0.22394999999999998,-0.42469999999999997,-0.3681,-0.1994,0.24559999999999998,0.30485,0.81745,-0.8189500000000001,0.43885],[7.071067811864696e-05,0.0004949747468305876,0.0008485281374238814,0.0015556349186103902,0.0025455844122715655,0.0025455844122715854,0.0027577164466275456,0.0030405591591021333,0.003181980515339506,0.0034648232278140937],['g-9','g-715','g-582','g-260','g-102','g-125','g-585','c-91','g-392','c-35'])],[('72 * D1',[0.45435000000000003,-0.00015,1.3475000000000001,-0.2434,-0.2429,-0.0374,0.487,-0.24080000000000001,0.26915,0.07725],[7.071067811868621e-05,0.00021213203435596425,0.002121320343559566,0.0025455844122715655,0.004242640687119289,0.004525483399593901,0.005798275605729679,0.008768124086713186,0.009121677477306479,0.009970205614730322],['c-24','g-370','g-543','g-472','g-259','g-24','c-43','g-423','g-378','g-740']),('48 * D1',[-0.5348999999999999,0.23025,0.0318,0.3365,0.01795,-0.51275,0.31415000000000004,-0.144,-0.19935,0.7099],[0.0004242640687119603,0.0006363961030678816,0.0009899494936611679,0.0009899494936611753,0.0014849242404917507,0.0036062445840513878,0.004030508652763309,0.004242640687119289,0.004737615433949857,0.006646803743153521],['g-150','g-460','c-80','g-238','g-609','g-458','g-291','g-446','g-78','c-30']),('72 * D2',[0.32489999999999997,-0.08285000000000001,0.2242,-0.64635,0.02065,0.3648,0.2491,0.54775,-0.29935,-0.6628000000000001],[0.0005656854249492149,0.0006363961030678914,0.001979898987322331,0.002192031021678291,0.0023334523779156074,0.0032526911934581135,0.003535533905932721,0.003747665940288682,0.003889087296526015,0.004525483399593955],['g-628','g-601','g-573','g-478','g-577','g-484','g-686','g-34','g-0','g-392']),('48 * D2',[0.036699999999999997,-0.6093999999999999,-0.2179,0.4527,0.33425,0.18665,0.09065000000000001,0.8172999999999999,-0.9582999999999999,0.4969],[0.0002828427124746222,0.0005656854249492542,0.0014142135623730963,0.0026870057685088986,0.004171930009000642,0.005303300858899111,0.006293250352560276,0.006505382386916227,0.006646803743153521,0.006929646455628187],['g-617','g-609','g-214','g-768','g-312','g-137','g-578','g-173','c-49','g-163'])],[('48 * D2',[-0.74125,-0.42835,-0.60535,-0.04075,-0.10545,-0.6195999999999999,0.151,-0.12795,-0.26465000000000005,-0.6951],[0.0006363961030679012,0.0006363961030679012,0.001060660171779783,0.0010606601717798223,0.0031819805153394665,0.0033941125496954466,0.0035355339059327407,0.0037476659402887014,0.003889087296526015,0.004808326112068543],['g-266','g-110','g-0','g-760','g-677','c-27','g-261','g-601','g-214','g-215']),('72 * D2',[0.41169999999999995,0.5556,0.43479999999999996,-0.59735,0.48785,0.0311,0.5804,-0.2511,0.2006,0.5766],[0.0002828427124746271,0.0012727922061358025,0.0015556349186104294,0.0038890872965259755,0.0045961940777125634,0.004666904755831215,0.005232590180780425,0.00664680374315356,0.007636753236814697,0.007778174593051951],['g-605','g-686','g-492','g-582','c-65','g-355','g-148','g-579','g-761','g-407'])],[('48 * D1',[0.45785,-0.1194,-0.3581,0.24669999999999997,0.07469999999999999,0.11305,-0.08345,-0.32575,0.40605,0.9484],[0.00021213203435598016,0.0005656854249492444,0.0012727922061357632,0.0012727922061357827,0.0016970562748477138,0.0017677669529663704,0.002333452377915605,0.0023334523779156243,0.0024748737341529184,0.003959797974644701],['g-580','g-264','g-241','g-145','g-201','g-108','g-34','g-647','c-99','c-30'])],[('72 * D2',[0.193,-0.53345,0.2243,0.72545,0.11755,-0.42655,0.30945,0.23010000000000003,0.17565,-0.36685],[0.00014142135623731355,0.0006363961030679012,0.0029698484809834867,0.0030405591591021333,0.0031819805153394665,0.003181980515339467,0.0034648232278140937,0.004101219330881975,0.0045961940777125634,0.004737615433949857],['g-383','c-51','c-97','g-455','g-545','g-284','g-337','c-95','g-351','g-567']),('72 * D1',[-0.00015,0.00015,0.5750500000000001,0.6252,0.00185,-0.20629999999999998,0.0739,-0.33445,0.05845,0.6591],[0.00021213203435596425,0.00021213203435596425,0.00021213203435601938,0.00042426406871188177,0.002616295090390226,0.0031112698372208003,0.003818376618407358,0.007000357133746795,0.007848885271170676,0.007919595949289324],['g-370','g-238','g-286','g-606','g-481','g-615','g-321','g-627','g-594','g-714'])],[('48 * D1',[-0.5892,-1.115,-0.37265,0.06275,-0.8995,-0.10525000000000001,0.0946,-0.13774999999999998,-0.7094499999999999,0.12845],[0.0008485281374238421,0.0028284271247461927,0.0028991378028648397,0.0028991378028648445,0.003959797974644622,0.004737615433949867,0.004949747468305837,0.0058689862838483455,0.009404520189781067,0.011384419177103427],['g-174','g-165','g-741','g-450','g-430','g-718','g-698','g-616','g-432','g-180'])],[('24 * D2',[0.2182,0.4098,0.5427,1.0675,-0.20779999999999998,0.19485,0.4314,0.3247,-0.06195,-0.44920000000000004],[0.0002828427124746271,0.0007071067811865482,0.0012727922061358025,0.002121320343559566,0.003959797974644681,0.004737615433949857,0.00777817459305203,0.009050966799187793,0.010111626970967624,0.016122034611053274],['g-312','g-267','g-162','g-70','c-43','g-747','g-743','g-517','g-396','g-551']),('48 * D1',[-0.00015,0.3258,-0.0717,0.2968,-0.7784,0.03895,-0.372,0.10125,-0.27764999999999995,0.5864499999999999],[0.00021213203435596425,0.00042426406871192107,0.0005656854249492346,0.0005656854249492542,0.0019798989873223505,0.0036062445840513925,0.0045254833995938765,0.00487903679018718,0.006858935777509501,0.008980256121069185],['g-370','g-12','g-329','g-622','g-545','g-460','g-330','g-34','c-2','g-261'])],[('48 * D1',[0.0872,-0.19590000000000002,-1.7595,0.28685,-0.26854999999999996,-0.2212,-0.25325,1.6844999999999999,-0.0179,0.13445000000000001],[0.0007071067811865482,0.002404163056034252,0.003535533905932819,0.0036062445840513878,0.0045961940777125634,0.00622253967444162,0.00756604255869605,0.007778174593051951,0.008626702730475881,0.009687362902255713],['g-559','g-75','g-85','g-437','c-71','g-733','g-389','g-190','g-339','g-616'])],[('24 * D2',[0.43379999999999996,1.0554999999999999,-0.52285,0.08505,0.1055,0.45865,0.58375,-0.0424,-0.0759,-0.5346],[0.0005656854249492149,0.0007071067811866267,0.001060660171779783,0.001909188309203684,0.0021213203435596446,0.0021920310216782916,0.0045961940777125235,0.00466690475583121,0.006505382386916237,0.00777817459305203],['g-8','g-407','g-742','g-187','g-619','c-87','c-24','g-516','c-12','g-125']),('72 * D1',[-0.57865,0.10815,-0.061149999999999996,0.8692,-0.3153,0.2027,-0.33555,-0.55135,-0.07150000000000001,0.4452],[7.071067811864696e-05,0.0003535533905932741,0.0010606601717798173,0.0011313708498985084,0.0016970562748477233,0.0018384776310850174,0.0024748737341529184,0.0044547727214752295,0.006646803743153551,0.0072124891681027755],['g-292','g-322','g-0','g-548','c-98','c-81','g-315','c-37','g-435','g-53'])],[('24 * D1',[0.17099999999999999,-0.2369,0.1001,0.2814,0.025,-0.016,-0.24275000000000002,-0.6345000000000001,0.9877,0.37329999999999997],[0.0005656854249492346,0.0012727922061358025,0.0022627416997969482,0.0024041630560342714,0.0028284271247461905,0.0043840620433565946,0.004454772721475249,0.004666904755831249,0.005656854249492385,0.005798275605729718],['g-354','g-390','g-166','g-490','g-435','g-525','g-327','c-24','g-122','g-224']),('72 * D2',[0.049350000000000005,0.04485,0.07085,0.1202,0.050850000000000006,-0.07255,-0.04315,-0.34765,0.23020000000000002,-0.20415],[0.0004949747468305827,0.0006363961030678914,0.0013435028842544395,0.002687005768508879,0.0028991378028648445,0.0033234018715767705,0.00487903679018718,0.005020458146424484,0.005515432893255071,0.007707463914933383],['g-273','g-591','g-648','g-641','g-116','g-534','g-191','g-196','g-111','g-623']),('72 * D1',[0.34095,0.15865,-0.1638,-0.14715,0.0847,-0.13690000000000002,0.7362500000000001,-0.7659499999999999,0.23370000000000002,-0.6326],[7.071067811868621e-05,0.00021213203435596054,0.0011313708498984691,0.0016263455967290568,0.003394112549695427,0.0036769552621700543,0.005303300858899072,0.006293250352560286,0.006646803743153541,0.006788225099390815],['g-718','g-333','g-13','g-403','g-35','c-77','g-444','g-546','g-278','g-591'])],[('48 * D1',[-0.0886,-0.9166,0.8382000000000001,-0.30525,0.39005,-0.11230000000000001,0.23055,0.18955,0.34465,-0.0256],[0.00014142135623731355,0.0014142135623730963,0.0015556349186104687,0.0017677669529663704,0.003889087296526015,0.006505382386916237,0.00756604255869605,0.00912167747730646,0.009970205614730322,0.012162236636408618],['g-437','g-658','g-687','g-679','g-277','g-116','g-494','g-191','g-155','g-338'])],[('48 * D1',[-0.84945,-0.35265,0.49235,-0.3304,-0.08835,-0.39435,-0.80125,-0.9002,-1.2325,-0.50935],[0.0007778174593051951,0.0012020815280171162,0.0019091883092037035,0.005374011537017758,0.005727564927611032,0.006717514421272207,0.0071417784899841285,0.009050966799187753,0.009192388155425205,0.009404520189781067],['g-770','g-522','g-197','g-122','g-25','c-96','g-466','g-627','g-458','g-708'])],[('72 * D1',[-0.046200000000000005,0.29705,-0.0166,-0.6094999999999999,-0.5295000000000001,-0.8896,-0.27290000000000003,-0.42005000000000003,1.0505,0.0991],[0.00042426406871192595,0.0006363961030679012,0.001555634918610405,0.0033941125496954466,0.005091168824543131,0.005515432893255012,0.0059396969619670125,0.006151828996322954,0.006363961030678855,0.008485281374238568],['g-612','c-98','g-719','g-477','g-203','g-555','g-487','g-753','g-309','g-508'])],[('48 * D1',[1.3255,-1.5095,1.144,4.8919999999999995,-0.3583,-0.75265,-1.893,1.1444999999999999,-1.893,-2.1305],[0.002121320343559566,0.002121320343559566,0.0028284271247461927,0.0028284271247465066,0.005656854249492385,0.006717514421272168,0.008485281374238578,0.009192388155425205,0.01131370849898477,0.014849242404917433],['g-681','g-446','g-763','g-144','g-81','g-290','g-186','g-676','g-717','g-209'])],[('72 * D1',[-0.25495,-0.4937,-0.43889999999999996,0.44984999999999997,-0.17935,-0.29505000000000003,0.06445000000000001,-0.27035,0.0834,-0.40565],[0.0004949747468306073,0.001414213562373057,0.0016970562748477233,0.002757716446627506,0.00332340187157678,0.0044547727214752685,0.00487903679018718,0.005303300858899111,0.005515432893255071,0.005586143571373738],['g-342','g-501','g-270','g-401','g-458','g-498','g-294','c-89','g-687','g-205']),('72 * D2',[-0.6353,0.1638,-0.54145,-0.3052,0.43420000000000003,-0.02965,-0.856,-0.0995,-0.43145,0.19135000000000002],[0.0004242640687119603,0.0008485281374238421,0.0014849242404917434,0.0022627416997969777,0.0024041630560342714,0.0024748737341529136,0.002545584412271605,0.0036769552621700443,0.0044547727214752295,0.008697413408594538],['g-498','g-258','g-2','g-329','c-28','g-87','g-657','g-134','g-488','g-501']),('24 * D2',[0.4948,-0.23345,-0.47665,-0.21284999999999998,0.0014500000000000001,0.5339499999999999,-0.28964999999999996,-0.46709999999999996,0.62,-0.53835],[0.00014142135623733317,0.0003535533905932741,0.0012020815280171162,0.0014849242404917629,0.0024748737341529167,0.003181980515339506,0.003747665940288721,0.003818376618407368,0.007495331880577363,0.007566042558696011],['g-571','g-585','g-343','g-32','g-356','g-458','g-592','g-189','g-532','c-56'])],[('48 * D1',[-0.2149,-0.028200000000000003,0.33855,-0.06065,0.8745499999999999,-0.2247,-0.1005,0.39975,-0.1877,-0.07780000000000001],[0.0007071067811865482,0.0011313708498984767,0.0019091883092036642,0.0020506096654409876,0.002616295090390173,0.002687005768508879,0.0039597979746446715,0.0044547727214752295,0.004949747468305837,0.005091168824543141],['c-43','g-487','g-23','g-178','c-9','c-14','g-38','g-513','c-98','g-473'])],[('48 * D2',[-0.04505,0.09025,0.093,0.03165,0.42085,-0.35075,0.1285,-0.12565,0.6146,0.1196],[0.00021213203435596054,0.0010606601717798223,0.0018384776310850174,0.001909188309203679,0.0020506096654409976,0.003181980515339467,0.0035355339059327407,0.0036062445840513977,0.0038183766184073284,0.004242640687119289],['g-393','g-311','g-475','g-649','g-736','g-436','g-80','g-611','g-185','g-461'])],[('24 * D1',[0.0366,-0.09445,0.35614999999999997,0.4679,0.347,0.11815,0.38139999999999996,0.16965,-0.12595,-0.8603000000000001],[0.00014142135623731355,0.0004949747468305778,0.0012020815280171162,0.0012727922061357632,0.0014142135623730963,0.0036062445840513977,0.004808326112068504,0.005727564927611032,0.0064346717087975895,0.007919595949289324],['g-569','g-727','g-70','c-35','g-654','g-74','g-348','g-649','g-213','c-18']),('24 * D2',[0.194,0.06345,0.37695,-0.06725,1.4445000000000001,-0.71695,0.45430000000000004,0.0045,-0.17170000000000002,-0.010400000000000001],[0.00014142135623731355,0.0020506096654409876,0.0023334523779155853,0.002616295090390222,0.003535533905932819,0.0044547727214752295,0.004808326112068504,0.006646803743153546,0.0072124891681027955,0.009333809511662428],['g-479','c-83','g-96','g-675','g-217','g-512','g-194','g-545','g-413','c-92'])],[('72 * D1',[-0.0123,-0.2449,-0.4185,0.9914000000000001,-0.01415,0.8543499999999999,1.1475,-0.17845,-0.46405,0.1142],[0.0011313708498984765,0.0016970562748477233,0.0025455844122715655,0.0038183766184073284,0.0038890872965260097,0.004171930009000563,0.004949747468305916,0.005020458146424484,0.005020458146424484,0.008768124086713186],['g-656','c-94','g-49','g-243','g-157','g-309','g-615','g-20','g-539','c-24']),('72 * D2',[-0.71155,-0.11005,-0.8863000000000001,0.2311,0.06720000000000001,-0.0208,-0.18485000000000001,0.28285,0.13785,0.4739],[0.00035355339059323483,0.0003535533905932741,0.0004242640687119603,0.0005656854249492542,0.0008485281374238519,0.0009899494936611655,0.0016263455967290568,0.0023334523779155853,0.0023334523779156243,0.0025455844122715655],['c-94','g-526','g-240','g-646','g-23','g-90','g-305','g-288','g-565','g-67'])],[('48 * D1',[0.1359,0.6517,-0.6073,-1.3625,-0.00035,0.20775,0.32075,-0.33809999999999996,-0.28915,-0.3094],[0.00042426406871192107,0.0004242640687119603,0.0005656854249492542,0.0007071067811864697,0.0013435028842544402,0.00332340187157678,0.0036062445840513878,0.003818376618407368,0.003889087296526015,0.004242640687119289],['g-178','g-702','g-736','g-671','g-506','g-334','g-160','g-564','g-157','g-743']),('24 * D1',[-0.6795,0.17559999999999998,0.51885,-0.0017000000000000001,0.54395,0.7294,-0.3505,0.10669999999999999,0.57635,-0.23175],[0.00028284271247458785,0.001979898987322331,0.0026162950903902515,0.00282842712474619,0.0028991378028648397,0.003111269837220859,0.0035355339059327407,0.004525483399593907,0.00601040764008562,0.007424621202458736],['g-207','c-85','g-674','g-679','g-508','c-36','g-259','g-58','g-738','c-92'])],[('72 * D2',[0.039599999999999996,-0.1729,-0.082,0.36560000000000004,0.29975,-0.1791,0.22660000000000002,-0.4978,0.58285,0.45325],[0.0002828427124746222,0.00042426406871192107,0.0005656854249492346,0.0022627416997969777,0.0028991378028648397,0.0035355339059327407,0.004808326112068524,0.005374011537017797,0.00685893577750954,0.0071417784899841285],['g-400','g-206','g-497','g-43','g-653','c-5','g-178','g-232','g-252','g-127']),('48 * D1',[-0.2025,0.7925,-0.22105000000000002,-0.15635,0.14215,-0.08195,0.28300000000000003,0.18185,-0.2607,0.36885],[0.00028284271247460747,0.00042426406871188177,0.0009192388155425087,0.0020506096654409776,0.0021920310216782916,0.002616295090390222,0.00311126983722082,0.0038890872965259954,0.0072124891681027755,0.00799030662740797],['g-502','g-446','g-753','g-712','c-68','g-108','g-119','g-346','g-171','g-401']),('24 * D1',[0.3316,-1.0470000000000002,-0.4593,-0.037000000000000005,0.47455,-0.08854999999999999,-0.26944999999999997,0.39465,1.104,0.69705],[0.0008485281374238421,0.0014142135623730963,0.0015556349186103902,0.001555634918610405,0.0017677669529663704,0.001909188309203674,0.003181980515339467,0.0033234018715768,0.004242640687119289,0.006010407640085699],['g-64','g-417','g-51','g-32','g-28','g-415','c-54','g-748','g-707','g-746']),('72 * D1',[0.3484,0.1182,-0.1131,0.65065,-0.06709999999999999,0.48414999999999997,-0.1694,-0.49905,-0.49305,0.1593],[0.0007071067811865482,0.0007071067811865482,0.0008485281374238519,0.0009192388155424891,0.0009899494936611655,0.0024748737341529184,0.0025455844122715655,0.0027577164466275456,0.0033234018715767605,0.003818376618407368],['g-527','g-456','g-745','c-63','g-331','g-574','g-349','g-479','g-545','g-570'])],[('24 * D2',[0.40495000000000003,0.14595000000000002,0.4605,-0.44705,-0.2034,0.50135,0.5888500000000001,0.16355,0.0461,-0.2208],[0.00021213203435598016,0.0004949747468305876,0.0007071067811865482,0.0028991378028648397,0.004384062043356583,0.006858935777509501,0.007707463914933382,0.009263098833543773,0.009475230867899738,0.010323759005323596],['g-43','g-146','c-26','g-252','g-111','g-687','g-641','c-38','g-766','c-93']),('72 * D2',[-0.18525,-0.17804999999999999,-0.3509,0.36495,-0.39245,-0.13195,0.09445,0.19165,0.44645,0.5111],[0.0003535533905932741,0.0013435028842544298,0.0016970562748477233,0.0026162950903902515,0.004313351365237936,0.0045961940777125435,0.006293250352560276,0.007848885271170676,0.008838834764831853,0.008909545442950459],['g-588','g-263','g-19','g-345','g-429','g-555','g-742','g-202','g-573','g-230']),('72 * D1',[-0.17304999999999998,0.36995,-0.743,0.7049000000000001,0.5467,0.4184,0.51745,-0.46365,1.1855,-0.12365000000000001],[0.0004949747468305876,0.001767766952966331,0.001979898987322272,0.0019798989873223505,0.0029698484809834867,0.0032526911934581135,0.003747665940288682,0.004030508652763348,0.004949747468305916,0.005020458146424494],['g-335','g-639','g-252','c-79','g-752','g-37','c-65','g-254','c-76','g-193'])],[('72 * D1',[-0.5327500000000001,0.06575,0.16544999999999999,-0.9493,0.2946,-0.0054,0.028499999999999998,0.39255,-1.0179999999999998,-0.5453],[7.071067811872547e-05,0.0010606601717798223,0.0012020815280171357,0.003252691193458153,0.003818376618407368,0.007636753236814713,0.007636753236814714,0.008555992052357225,0.009899494936611674,0.011172287142747477],['g-205','g-397','g-302','g-105','g-764','g-104','c-78','g-566','g-224','g-453']),('48 * D2',[-0.6056,0.6189,-0.0339,-0.50805,-0.34104999999999996,0.11255,-0.9282,-0.052099999999999994,0.45999999999999996,-0.8693],[0.00028284271247458785,0.00042426406871188177,0.0012727922061357827,0.001767766952966331,0.0024748737341529184,0.0028991378028648397,0.004101219330881995,0.004949747468305832,0.005232590180780464,0.00622253967444164],['g-596','g-260','g-453','g-500','g-598','c-58','g-599','g-79','g-556','g-316'])],[('48 * D1',[-0.5928,0.11324999999999999,0.07885,0.30055,1.1155,-0.6097,-3.114,0.36745,1.1880000000000002,0.7665500000000001],[0.00042426406871188177,0.0013435028842544395,0.0027577164466275356,0.003181980515339467,0.004949747468305759,0.005515432893255091,0.005656854249492385,0.005727564927611032,0.007071067811865481,0.007566042558696089],['c-88','g-180','g-738','c-79','g-665','g-116','g-51','g-193','g-761','g-315'])],[('48 * D1',[0.45085,0.06290000000000001,0.15400000000000003,-0.8069500000000001,-0.33485,0.93015,0.6517,0.0547,0.44645,0.55815],[0.00021213203435598016,0.0002828427124746173,0.0012727922061357827,0.002757716446627467,0.0033234018715767605,0.003747665940288682,0.004666904755831249,0.004949747468305832,0.005020458146424484,0.005727564927611032],['g-687','c-1','g-206','g-354','g-60','g-504','c-96','g-615','c-81','g-35'])],[('48 * D1',[-0.5702,-0.3124,-0.32655,0.6131500000000001,-0.31235,-0.01845,0.223,-0.0741,-0.2123,-0.5242],[0.00028284271247466634,0.0007071067811865482,0.0016263455967290372,0.0033234018715768,0.0036062445840513878,0.005868986283848343,0.006363961030678933,0.010606601717798213,0.011596551211459378,0.01244507934888328],['c-81','g-26','g-37','g-686','c-97','g-376','g-124','c-72','g-210','g-283'])],[('48 * D1',[0.08785,0.02935,0.75585,0.30625,0.0959,-0.34105,-0.44545,0.46199999999999997,0.09765,0.0032],[0.0009192388155425087,0.0010606601717798197,0.0010606601717798615,0.002050609665440958,0.0021213203435596446,0.0021920310216782916,0.0023334523779156243,0.0032526911934581135,0.0036062445840513878,0.0043840620433565946],['g-181','g-745','g-260','g-139','g-512','g-356','g-184','g-97','g-103','g-234'])],[('48 * D1',[0.12165000000000001,-0.4337,-1.4740000000000002,0.19695000000000001,-0.14005,-0.09655,-0.15305000000000002,-0.36465000000000003,0.20365,0.5784],[0.000777817459305205,0.0009899494936611753,0.0014142135623730963,0.002616295090390232,0.0028991378028648397,0.003464823227814084,0.004454772721475249,0.0045961940777125634,0.0045961940777125634,0.005091168824543131],['g-589','g-66','g-371','g-9','g-562','g-26','g-22','g-243','g-429','g-698'])],[('72 * D2',[0.1825,0.75095,0.40635,0.7045,-0.055900000000000005,-0.65545,0.33035,-0.0639,0.6690499999999999,0.98905],[0.0005656854249492346,0.0013435028842544493,0.0014849242404917434,0.0016970562748476843,0.007778174593052025,0.008697413408594519,0.009263098833543773,0.009758073580374355,0.010677312395916868,0.011667261889578005],['c-35','g-705','c-7','g-543','g-612','g-38','g-710','g-30','g-19','g-651']),('48 * D2',[0.69935,-0.3158,-0.7302,0.2403,0.15545,0.09425,0.05015,0.01595,-0.3175,0.70875],[0.00021213203435594088,0.0007071067811865482,0.0018384776310850566,0.001979898987322331,0.0033234018715767605,0.005303300858899111,0.005303300858899111,0.005444722215136417,0.0059396969619669735,0.006576093065034952],['g-414','g-547','g-438','g-139','g-97','g-79','g-195','g-65','g-77','g-545'])],[('72 * D1',[-0.00015,0.02495,-0.29705000000000004,-0.8447,0.181,-0.0191,0.2678,-0.23395,2.036,-0.4899],[0.00021213203435596425,0.0009192388155425136,0.0012020815280171162,0.001979898987322272,0.0028284271247461927,0.00311126983722081,0.0035355339059327407,0.0037476659402887014,0.005656854249492385,0.007919595949289324],['g-370','g-206','g-566','g-419','g-117','g-380','g-736','g-623','g-69','g-192'])],[('48 * D1',[0.38634999999999997,-0.5325,-0.9198,0.32445,1.1385,1.0875,-0.6225,-1.011,0.98745,0.883],[0.0012020815280171162,0.0016970562748476843,0.0021213203435596446,0.004737615433949857,0.004949747468305759,0.004949747468305759,0.005232590180780425,0.005656854249492385,0.005727564927611032,0.0059396969619669735],['g-475','g-260','g-678','g-368','g-52','g-494','g-717','g-413','g-432','g-511'])],[('48 * D1',[0.88845,-0.40459999999999996,0.17485,0.6119,0.023100000000000002,0.00225,-0.5799,0.4426,0.17135,0.4668],[7.071067811864696e-05,0.0009899494936611753,0.0016263455967290568,0.0018384776310849781,0.0018384776310850248,0.0023334523779156065,0.0029698484809834867,0.0032526911934581135,0.0033234018715767605,0.0033941125496954466],['c-59','g-225','g-448','c-78','c-31','c-42','c-85','c-66','g-550','c-13'])],[('48 * D1',[-0.64125,0.32294999999999996,-0.2097,-0.26944999999999997,-0.04765,0.3391,0.5342,-0.23745,0.4533,0.65125],[0.0004949747468305288,0.0012020815280171162,0.0014142135623730963,0.0019091883092036642,0.0037476659402887014,0.004242640687119289,0.004666904755831171,0.004879036790187171,0.004949747468305837,0.00530330085889915],['g-487','g-167','g-110','g-435','g-176','c-22','g-389','g-14','c-36','g-554'])],[('48 * D1',[0.28955,-0.2774,-0.33755,0.25415,0.93965,0.0143,0.3908,0.7677,0.31305,0.11935],[0.00021213203435598016,0.0012727922061357632,0.0013435028842544493,0.0023334523779156243,0.0031819805153394274,0.0036769552621700456,0.004242640687119289,0.004525483399593955,0.005020458146424484,0.0071417784899841285],['g-16','g-365','g-222','g-339','g-735','g-538','g-574','g-159','g-440','g-726'])],[('48 * D1',[-1.2545,0.4974,-0.1082,0.5076,0.37165000000000004,-0.26795,0.19455,-0.8571,-0.3301,0.39615],[0.002121320343559723,0.0022627416997969383,0.002687005768508879,0.003111269837220859,0.0033234018715767605,0.003747665940288721,0.004171930009000642,0.004949747468305837,0.004949747468305837,0.005868986283848326],['c-38','c-41','c-14','g-341','c-75','c-95','g-386','c-65','g-247','g-479'])],[('48 * D1',[-0.7737499999999999,-0.034,0.25555,-0.99715,0.8100499999999999,-0.2626,-0.32155,-0.25175,-0.46185,0.6823],[0.0004949747468306073,0.0018384776310850222,0.0024748737341528794,0.002474873734152958,0.0030405591591022123,0.005232590180780425,0.005303300858899072,0.006293250352560266,0.006434671708797581,0.006929646455628187],['g-633','g-639','g-348','g-29','g-179','g-280','g-105','g-567','g-367','c-66'])],[('72 * D2',[0.82155,-1.1025,-0.58145,-0.3479,-0.10794999999999999,-0.3342,0.31074999999999997,0.2449,-0.43465,-0.6311],[7.071067811864696e-05,0.0007071067811864697,0.0026162950903902515,0.006788225099390855,0.008131727983645294,0.008626702730475872,0.008838834764831853,0.009192388155425127,0.009970205614730322,0.012586500705120572],['g-613','g-708','g-642','g-505','g-611','c-54','g-726','c-24','g-589','g-654']),('72 * D1',[-0.07505,-0.1066,0.82375,-0.3516,-0.1755,-0.1711,0.1609,0.33655,0.2673,0.055999999999999994],[7.071067811865677e-05,0.0011313708498984691,0.0016263455967290372,0.0016970562748477233,0.001979898987322331,0.0035355339059327407,0.003959797974644681,0.004171930009000642,0.004384062043356583,0.004666904755831215],['g-144','g-402','g-253','g-197','g-760','g-697','g-195','g-165','g-542','g-338'])],[('48 * D1',[-0.4153,0.269,0.7352000000000001,-0.2801,-0.63035,0.28205,0.5367,-0.4229,-0.8417,-0.0513],[0.0007071067811865482,0.0007071067811865482,0.0025455844122715264,0.0029698484809834867,0.003181980515339506,0.004313351365237936,0.005091168824543131,0.005656854249492385,0.006363961030678855,0.006929646455628168],['g-752','g-155','g-471','g-231','g-147','g-581','g-687','g-459','g-442','g-529'])],[('48 * D1',[1.5194999999999999,-1.1145,0.40905,-4.871,-0.1133,1.1684999999999999,-9.8775,-9.8775,-9.8775,-9.8775],[0.0007071067811866267,0.002121320343559723,0.0034648232278140546,0.004242640687118818,0.007636753236814716,0.009192388155425205,0.010606601717798614,0.010606601717798614,0.010606601717798614,0.010606601717798614],['g-279','g-485','g-628','g-669','g-520','g-740','c-34','c-39','c-43','c-33'])],[('48 * D1',[-0.3788,0.38105,-0.20240000000000002,1.598,-1.5590000000000002,1.2195,0.87325,-0.16699999999999998,0.0302,0.8302499999999999],[0.00014142135623733317,0.0020506096654409976,0.002687005768508879,0.0028284271247461927,0.004242640687119289,0.009192388155425047,0.009828784258493027,0.010182337649086283,0.010182337649086286,0.011808683245815377],['c-57','g-326','g-748','g-462','g-507','g-569','g-306','g-146','g-25','g-401'])],[('48 * D1',[0.944,-0.26475000000000004,-0.42395,-0.45455,-0.543,1.0825,-0.0114,0.15284999999999999,-0.13155,-0.11905],[0.0005656854249492542,0.0010606601717798223,0.0012020815280171554,0.0014849242404917434,0.0015556349186103902,0.002121320343559723,0.00282842712474619,0.0036062445840514073,0.004454772721475249,0.004454772721475249],['g-489','g-646','g-268','g-609','g-364','c-25','g-62','g-607','g-27','g-575'])],[('72 * D1',[-0.00015,0.74595,-0.11315,-0.08015,-0.31975,-0.3815,-0.9884999999999999,1.225,1.545,-0.36415],[0.00021213203435596425,0.0006363961030678226,0.0007778174593051951,0.0009192388155425087,0.0012020815280171162,0.0012727922061358025,0.0022627416997969383,0.0028284271247461927,0.004242640687119289,0.005303300858899111],['g-370','g-650','g-97','g-224','g-109','c-84','g-405','g-38','g-22','g-468']),('72 * D2',[0.14345000000000002,-0.21685,0.8135,0.09175,-0.05345,0.44645,0.22615,0.1936,0.4931,0.6576500000000001],[0.00021213203435596054,0.0004949747468305876,0.0005656854249491757,0.001909188309203684,0.0031819805153394617,0.003889087296526015,0.0045961940777125634,0.006081118318204306,0.007071067811865481,0.008273149339882558],['g-580','g-22','g-214','g-622','g-672','g-585','g-718','c-76','g-4','g-438'])],[('48 * D1',[0.469,0.32845,-0.3207,-0.36395,0.505,-0.6224000000000001,-0.23265,-2.2285000000000004,-0.7024999999999999,0.24565],[0.0011313708498984691,0.003181980515339467,0.004101219330881955,0.0044547727214752295,0.004808326112068465,0.005091168824543131,0.006151828996322953,0.006363961030678855,0.006363961030678934,0.006576093065034913],['g-31','g-741','g-253','g-517','g-123','g-509','g-662','c-69','c-50','g-90'])],[('24 * D1',[0.38885000000000003,0.39735,-0.17554999999999998,1.6295,0.59615,0.3064,0.69665,0.43015000000000003,-1.2005,-0.035949999999999996],[0.00021213203435598016,0.000494974746830568,0.005020458146424504,0.006363961030678855,0.007707463914933382,0.008343860018001284,0.010111626970967616,0.011242997820866085,0.012020815280171397,0.01209152595828996],['g-61','g-397','g-328','g-55','c-81','g-611','c-2','c-90','g-320','g-24']),('48 * D2',[0.32330000000000003,-0.1631,0.6089,0.7199,-0.57995,0.05975,-1.2785,0.4941,0.22999999999999998,0.68675],[0.00014142135623733317,0.00042426406871192107,0.001979898987322272,0.0022627416997969383,0.0044547727214752295,0.00544472221513642,0.006363961030678855,0.007636753236814696,0.007636753236814716,0.008273149339882636],['g-430','g-681','g-751','g-733','g-157','g-300','g-753','c-75','g-324','g-685'])],[('48 * D1',[0.2306,0.2581,-0.7178,0.5439,0.9903500000000001,-0.24485,-0.30945,0.4952,-0.0097,0.6448499999999999],[0.0009899494936611753,0.0011313708498985084,0.0012727922061358025,0.0021213203435596446,0.0023334523779155853,0.0038890872965259954,0.003889087296526015,0.00466690475583121,0.004808326112068524,0.00487903679018719],['g-763','g-258','c-51','c-74','g-607','c-26','c-59','g-210','g-382','g-58'])],[('48 * D1',[0.16425,0.15315,-0.65045,-0.3089,-0.2052,0.2801,0.1012,0.3009,-0.86955,-0.07750000000000001],[7.071067811864696e-05,0.0017677669529663704,0.0023334523779155853,0.0028284271247461927,0.003818376618407348,0.003818376618407368,0.004242640687119289,0.0045254833995938765,0.005303300858899072,0.005798275605729689],['g-379','g-675','c-47','g-677','g-566','c-27','g-652','c-72','g-182','g-136'])],[('48 * D1',[-0.21295,0.017550000000000003,-0.82515,-0.9293,0.2744,0.18530000000000002,0.2868,-0.019200000000000002,-0.3002,1.1835],[0.0007778174593051951,0.0007778174593052025,0.0017677669529664096,0.0022627416997969383,0.0025455844122715655,0.004101219330881975,0.004101219330881995,0.004525483399593903,0.006222539674441601,0.006363961030678855],['g-723','g-130','g-598','g-385','g-546','g-663','g-769','c-56','g-322','g-80'])],[('48 * D1',[-3.5675,1.1355,0.2827,-0.43165,-0.73315,-1.1935,-1.0445,0.0030499999999999998,-0.6032,-0.0252],[0.0007071067811864697,0.002121320343559723,0.0022627416997969383,0.003889087296526015,0.007000357133746834,0.007778174593052108,0.007778174593052108,0.009121677477306462,0.010323759005323634,0.010465180361560902],['g-250','g-66','c-68','g-265','g-245','g-107','g-427','g-652','g-687','g-209'])],[('48 * D1',[0.08685000000000001,0.79725,-0.6326499999999999,-0.1098,-0.273,-0.00375,-0.7483500000000001,-0.3404,-0.3297,-0.78645],[7.071067811865677e-05,0.00021213203435594088,0.001060660171779783,0.0014142135623730963,0.0016970562748477233,0.002333452377915607,0.0034648232278140937,0.0035355339059327407,0.004384062043356583,0.004737615433949818],['c-96','g-125','g-328','g-64','g-454','c-95','g-97','g-178','c-68','g-448'])],[('72 * D1',[0.24309999999999998,-0.62935,-0.30210000000000004,-0.06275,0.2066,0.30495,-0.40415,1.2945,-0.0747,0.06520000000000001],[0.0008485281374238617,0.0013435028842544493,0.0016970562748477233,0.002333452377915605,0.004242640687119289,0.005868986283848326,0.005868986283848326,0.006363961030678855,0.008909545442950499,0.00933380951166243],['g-684','g-708','g-737','g-100','c-91','g-751','g-759','g-199','g-664','g-404']),('24 * D2',[-0.0766,-0.10525,0.8579,0.24480000000000002,0.93405,-0.4578,-0.0403,0.26339999999999997,0.21184999999999998,0.46335],[0.00042426406871192107,0.0016263455967290568,0.0018384776310849781,0.002404163056034252,0.003181980515339506,0.005091168824543131,0.0050911688245431404,0.005232590180780464,0.005586143571373719,0.005727564927611032],['g-607','g-714','g-565','c-46','g-240','g-481','g-664','g-331','g-26','g-587'])],[('48 * D1',[-0.0849,-0.7301,0.16225,-0.7151,0.70425,-0.54275,-0.6531,-0.5851,0.18635000000000002,-0.3604],[0.00042426406871193083,0.0005656854249492542,0.0010606601717798223,0.0012727922061358025,0.002192031021678291,0.004313351365237936,0.004808326112068465,0.005374011537017719,0.006576093065034894,0.006646803743153521],['g-91','g-320','g-480','g-31','g-609','g-34','g-169','g-745','g-651','g-134'])],[('48 * D1',[-0.84495,0.12079999999999999,-0.89895,0.53315,-0.5535,-0.06105,0.8855999999999999,0.78305,1.2915,0.4543],[0.00021213203435594088,0.0018384776310850272,0.0026162950903902515,0.0030405591591021333,0.0032526911934580745,0.0033234018715767705,0.0033941125496954466,0.00487903679018719,0.006363961030679012,0.006505382386916227],['g-652','g-511','c-95','g-755','g-427','c-83','g-417','g-175','g-570','c-20'])],[('24 * D2',[-0.035750000000000004,0.5296,-0.09135,0.36095,-1.084,0.24185,-0.17965,-0.6558999999999999,0.0082,0.27555],[7.071067811865187e-05,0.0014142135623730963,0.0017677669529663704,0.0017677669529663704,0.0028284271247461927,0.0036062445840513878,0.004737615433949857,0.007353910524340148,0.007778174593052023,0.007848885271170676],['g-231','g-694','g-131','g-470','g-211','g-236','g-740','g-235','g-173','g-394']),('72 * D1',[0.7516,0.635,1.1555,0.2974,0.3749,-0.5104,-0.89705,0.37995,0.786,0.7563],[0.00014142135623737242,0.00028284271247458785,0.0007071067811864697,0.0011313708498985084,0.0012727922061358025,0.0028284271247461927,0.0030405591591021333,0.0036062445840513878,0.003959797974644622,0.004101219330881995],['g-17','g-673','g-10','g-526','g-329','g-99','g-589','g-38','g-285','g-406']),('48 * D1',[0.6083000000000001,-0.23615,-0.3438,-0.49770000000000003,-0.0639,0.16225,0.1509,-0.71705,-0.1494,0.11095],[0.000989949493661136,0.0012020815280171162,0.0018384776310850174,0.001979898987322311,0.0021213203435596446,0.0021920310216782916,0.0028284271247461927,0.0034648232278140937,0.0035355339059327407,0.004030508652763319],['c-80','g-398','g-146','g-724','g-291','g-88','g-179','g-192','g-138','g-171']),('72 * D2',[0.20295000000000002,0.6639999999999999,0.1114,0.6611499999999999,-0.5081,0.363,0.0172,-0.0409,-0.0684,0.6484000000000001],[0.0004949747468305876,0.0011313708498984299,0.0018384776310850174,0.0020506096654409976,0.0026870057685088986,0.00311126983722082,0.0032526911934581187,0.0035355339059327407,0.004242640687119289,0.004949747468305837],['g-131','g-300','g-378','g-243','g-519','g-184','g-12','g-620','g-755','g-613'])],[('48 * D1',[-0.45175,-0.17909999999999998,0.4131,0.37929999999999997,-0.16705,-0.42765,0.5017499999999999,-0.19205,0.31675,0.31105],[0.0007778174593051951,0.0011313708498984889,0.0015556349186104294,0.0022627416997969777,0.0028991378028648397,0.0044547727214752685,0.005161879502661778,0.005727564927611032,0.006293250352560247,0.007000357133746834],['g-219','g-405','c-70','g-90','g-540','c-34','g-581','c-2','c-25','c-96'])],[('72 * D2',[0.12465000000000001,-0.37875000000000003,0.9891,-0.10915,0.10215,-0.1391,0.43775,-0.3702,-0.57325,0.45505],[7.071067811865677e-05,0.0014849242404917826,0.0018384776310849781,0.002192031021678301,0.0024748737341529184,0.0035355339059327407,0.0045961940777125634,0.008343860018001244,0.009404520189781067,0.009404520189781067],['g-325','g-385','g-191','g-684','c-64','g-514','g-485','g-440','g-202','g-64']),('48 * D2',[0.8632500000000001,-0.28225,0.42874999999999996,0.07785,-0.2996,-0.28925,0.22025,-0.27564999999999995,-0.12365000000000001,-0.603],[0.0007778174593051951,0.0014849242404917434,0.0016263455967290765,0.0017677669529663704,0.0028284271247461927,0.0030405591591021333,0.003889087296526015,0.003889087296526015,0.0038890872965260245,0.003959797974644701],['g-762','c-60','g-565','g-226','g-299','g-42','g-626','g-628','g-727','g-51']),('72 * D1',[-0.5905,0.00015,-0.25705,0.2243,-0.5382,-0.1352,0.22139999999999999,-0.6772,-0.3564,0.093],[0.00014142135623729392,0.00021213203435596425,0.0016263455967290372,0.0024041630560342714,0.0045254833995938765,0.004808326112068524,0.0059396969619669926,0.00622253967444164,0.006363961030678933,0.006505382386916237],['g-506','g-370','g-323','g-147','g-266','g-204','g-536','g-541','g-95','g-704'])],[('48 * D1',[0.2308,-0.19540000000000002,0.52155,-0.6365000000000001,-1.48,-0.37875000000000003,-0.35350000000000004,1.5505,0.79365,-0.43984999999999996],[0.00042426406871192107,0.0005656854249492346,0.0016263455967291157,0.0022627416997969383,0.0028284271247461927,0.0034648232278140546,0.003959797974644662,0.006363961030678855,0.00685893577750954,0.007000357133746795],['c-97','g-713','c-46','g-422','g-710','g-619','g-214','g-476','c-49','g-407'])],[('72 * D2',[0.01815,-0.12785000000000002,-0.32465,0.47225,-0.5426,3.4690000000000003,-0.01095,0.25675000000000003,-1.2585,-0.29510000000000003],[7.071067811865432e-05,0.0037476659402887014,0.005161879502661778,0.005303300858899072,0.007495331880577442,0.008485281374238578,0.008697413408594536,0.008980256121069147,0.0106066017177983,0.010748023074035556],['g-601','g-315','g-736','g-265','g-668','g-208','g-768','g-40','g-324','g-322']),('72 * D1',[0.08990000000000001,-0.08990000000000001,-0.0002,-1.1555,0.5908,0.051500000000000004,-0.04965,0.194,-0.6496,0.661],[0.0002828427124746173,0.0002828427124746173,0.000282842712474619,0.0007071067811864697,0.0012727922061358025,0.0021213203435596394,0.0031819805153394665,0.003818376618407368,0.004384062043356583,0.005374011537017719],['g-238','g-307','g-266','g-719','g-641','g-498','g-683','g-394','g-327','g-106'])],[('48 * D1',[-0.03555,0.0691,-0.4312,-0.58445,-0.27575000000000005,-1.129,0.30155,0.08549999999999999,-0.40435,0.24085],[0.00021213203435596541,0.0012727922061357927,0.0026870057685088595,0.0028991378028648397,0.003181980515339467,0.004242640687119289,0.005161879502661817,0.007212489168102785,0.007707463914933383,0.007990306627407991],['g-363','g-387','g-119','c-33','g-222','g-318','g-696','g-364','c-56','g-736'])],[('48 * D1',[-1.0385,0.3809,0.26234999999999997,0.33730000000000004,0.6910499999999999,-0.78545,0.22044999999999998,0.23375,-0.7106,0.21905000000000002],[0.0007071067811864697,0.0011313708498984691,0.0012020815280171162,0.0016970562748477233,0.002192031021678291,0.003889087296526054,0.004030508652763328,0.004313351365237955,0.004384062043356583,0.007707463914933363],['g-619','g-586','g-281','g-311','g-599','g-145','g-217','g-354','g-727','g-747'])],[('48 * D1',[-0.34195,-0.41054999999999997,0.14715,-0.4298,-0.3275,0.8217,0.49505,-0.0336,0.30810000000000004,-0.32315],[0.0010606601717798223,0.0012020815280171162,0.002333452377915605,0.0029698484809834867,0.0033941125496954466,0.004384062043356583,0.0045961940777125634,0.005232590180780452,0.005374011537017758,0.006151828996322993],['g-313','g-333','g-61','c-1','g-165','g-332','g-441','c-95','g-295','g-120'])],[('48 * D1',[0.3325,0.1935,-0.23954999999999999,-1.2854999999999999,0.9917,0.029,0.4492,0.4651,-0.9417,-0.0643],[0.0009899494936611753,0.00155563491861041,0.001909188309203684,0.002121320343559566,0.0024041630560342323,0.002404163056034262,0.0029698484809834867,0.004101219330881995,0.004666904755831171,0.005798275605729694],['g-42','c-79','g-327','g-414','g-43','g-601','g-185','g-193','c-40','g-52'])],[('48 * D1',[-0.24485,-0.14395000000000002,0.28675,0.2349,-0.0363,-0.374,0.6265499999999999,-0.20905,-0.18080000000000002,0.8392],[0.00021213203435596054,0.00021213203435596054,0.0009192388155424891,0.0021213203435596446,0.0021213203435596446,0.005091168824543131,0.006858935777509462,0.007000357133746815,0.008061017305526637,0.00820243866176391],['c-9','g-24','g-415','g-19','g-692','g-148','g-555','g-381','g-106','c-75'])],[('72 * D2',[-0.12025,-0.3426,-0.45835,0.46435000000000004,-0.3021,-0.30315,-0.70555,-0.10239999999999999,0.63935,0.0253],[0.0003535533905932741,0.0009899494936611753,0.0013435028842544493,0.0023334523779156243,0.0028284271247461927,0.008555992052357225,0.008980256121069107,0.00933380951166243,0.009828784258493027,0.011030865786510139],['g-579','g-740','g-160','g-217','g-291','g-435','g-565','g-15','g-475','g-723']),('72 * D1',[-0.28615,-0.2883,-0.5184500000000001,0.4719,0.30484999999999995,-0.07635,-0.3972,0.84125,0.044950000000000004,0.3012],[7.071067811864696e-05,0.0005656854249492542,0.0009192388155424891,0.0022627416997969383,0.0024748737341529184,0.004737615433949867,0.005374011537017758,0.006717514421272168,0.007707463914933368,0.007778174593051991],['g-121','c-96','g-669','g-338','g-454','g-295','g-628','c-87','c-98','g-308']),('48 * D1',[-0.1369,0.14934999999999998,0.85905,-1.185,-0.13979999999999998,1.4264999999999999,-0.6305499999999999,0.20065,0.2827,0.13385],[0.0002828427124746271,0.0004949747468305876,0.0014849242404917434,0.0028284271247461927,0.0031112698372208003,0.003535533905932819,0.0047376154339498966,0.006293250352560266,0.006505382386916227,0.007000357133746834],['g-505','g-286','c-17','g-552','g-550','c-76','c-37','g-494','g-433','g-74']),('24 * D2',[-0.6577,-0.13,-0.16155,0.6718,-0.25035,0.70685,-0.2238,0.692,0.25805,1.3205],[0.0007071067811865482,0.0007071067811865482,0.0016263455967290765,0.0019798989873223505,0.0030405591591021533,0.0033234018715768,0.003959797974644662,0.004525483399593955,0.00487903679018719,0.004949747468305916],['g-169','g-331','g-736','g-432','g-206','c-60','g-639','g-58','g-605','g-67'])],[('48 * D1',[0.9172,-1.1825,-1.5505,-3.2279999999999998,-2.001,-1.827,2.495,-1.422,-3.0785,0.5036],[0.00042426406871188177,0.0007071067811866267,0.003535533905932662,0.004242640687119132,0.004242640687119289,0.004242640687119289,0.007071067811865324,0.008485281374238578,0.009192388155425047,0.00933380951166242],['g-740','g-89','g-212','g-313','g-456','g-114','g-468','g-172','g-143','g-452'])],[('24 * D1',[-0.762,-0.36785,-0.40745,-0.4644,-1.1695,-0.0418,-0.39615,0.0027,-0.5747,-0.46725],[0.0014142135623730963,0.0019091883092036642,0.002050609665440958,0.0021213203435596446,0.002121320343559723,0.002404163056034262,0.0024748737341529184,0.0038183766184073566,0.005232590180780503,0.006576093065034874],['g-35','g-470','g-640','g-621','g-192','g-481','g-27','g-408','g-569','g-190']),('48 * D1',[-0.25825,0.18305,0.1597,-0.2039,0.058399999999999994,0.9214,-0.33840000000000003,-0.2061,0.6089,-0.37434999999999996],[0.0003535533905932741,0.0014849242404917629,0.0029698484809835063,0.0032526911934581135,0.0032526911934581187,0.0035355339059327407,0.00466690475583121,0.005091168824543131,0.00509116882454321,0.005444722215136405],['g-81','c-6','g-495','g-242','g-610','g-322','g-693','g-611','g-1','g-373'])],[('24 * D1',[0.33640000000000003,1.874,0.0012,-0.07350000000000001,0.45289999999999997,-0.27135,0.2669,-0.5215000000000001,-0.14950000000000002,0.29885],[0.0009899494936611753,0.0014142135623730963,0.001697056274847714,0.00311126983722081,0.00311126983722082,0.0034648232278140937,0.003676955262170074,0.004666904755831249,0.00622253967444162,0.007707463914933383],['g-353','g-277','g-207','g-232','g-719','g-389','g-399','g-512','g-143','g-22'])],[('48 * D1',[1.944,-0.09659999999999999,1.5615,-1.3585,-0.40349999999999997,-0.26105,-0.8862,-0.36824999999999997,-0.16065000000000002,0.453],[0.0014142135623730963,0.0016970562748477138,0.006363961030679012,0.007778174593052108,0.008485281374238538,0.008555992052357225,0.009475230867899793,0.010394469683442242,0.012091525958289946,0.012162236636408611],['g-22','c-62','g-294','c-59','g-416','g-346','g-431','g-600','g-352','g-184'])],[('24 * D2',[-0.2554,-0.46099999999999997,-0.00955,-0.5446500000000001,-0.7822,-0.38765,0.05995,-0.14745,-0.66945,-0.41759999999999997],[0.0007071067811865482,0.0011313708498984691,0.0014849242404917494,0.0024748737341528794,0.002545584412271605,0.0030405591591021333,0.004171930009000627,0.004171930009000642,0.0045961940777125235,0.005374011537017758],['c-90','g-539','g-693','g-30','g-174','g-34','g-729','g-56','g-326','g-506']),('48 * D1',[-0.22685,-1.6885,-1.165,-0.10125,-0.05205,0.4324,-0.76915,-2.182,0.53765,-0.92755],[0.0006363961030679012,0.002121320343559566,0.0028284271247461927,0.003464823227814084,0.004171930009000632,0.004242640687119289,0.005161879502661856,0.005656854249492385,0.005727564927611032,0.007000357133746834],['g-119','c-40','g-741','g-88','c-51','g-68','g-75','c-18','c-88','g-673'])],[('48 * D1',[-0.34435,0.00155,0.20279999999999998,1.0455,1.0645,-0.10835,0.6852,-0.46314999999999995,-0.4437,-0.5106999999999999],[7.071067811864696e-05,0.0021920310216782973,0.0029698484809835063,0.004949747468305759,0.004949747468305916,0.005727564927611032,0.008061017305526618,0.008131727983645304,0.008626702730475872,0.009616652224137007],['c-30','g-702','g-603','g-251','g-43','g-490','g-504','g-32','g-648','g-272'])],[('48 * D1',[0.9165000000000001,-1.0205,-0.3083,-0.7121,0.2011,-0.4663,-0.43420000000000003,-0.45785,-0.33240000000000003,-0.80515],[0.0005656854249492542,0.0007071067811864697,0.001979898987322311,0.0024041630560342323,0.005374011537017758,0.0059396969619669735,0.007353910524340109,0.008414570696119892,0.013152186130069788,0.01378858223313765],['c-39','g-117','g-635','g-321','c-95','c-8','g-487','g-354','g-699','g-170'])],[('48 * D1',[-0.14615,0.9278500000000001,0.6848000000000001,-0.4723,0.2212,0.5544,0.6231,0.3196,-0.19979999999999998,0.17365000000000003],[0.0003535533905932741,0.0007778174593051951,0.0019798989873223505,0.0022627416997969383,0.0025455844122715655,0.0026870057685088986,0.0026870057685088986,0.0032526911934581135,0.0033941125496954275,0.004454772721475249],['g-160','g-351','g-81','c-35','g-392','c-91','c-28','c-69','g-382','c-17'])],[('24 * D1',[0.7436499999999999,-0.19435000000000002,0.14045,0.0072,-0.21955,-0.6031,0.1064,0.41925,0.28995,-0.6276999999999999],[0.00035355339059331333,0.0004949747468305876,0.0007778174593051951,0.0008485281374238568,0.0017677669529663704,0.0021213203435596446,0.0022627416997969482,0.004030508652763309,0.00487903679018719,0.005374011537017719],['g-134','c-45','c-47','g-195','g-157','g-521','g-335','g-597','g-142','g-315']),('72 * D2',[0.022699999999999998,0.4405,-0.18445,-0.33955,-0.3388,0.13385,0.17415,-0.31635,0.0081,-0.40490000000000004],[0.0007071067811865457,0.000989949493661136,0.0014849242404917434,0.0014849242404917434,0.0022627416997969383,0.0026162950903902125,0.0026162950903902125,0.004030508652763309,0.004101219330881975,0.004525483399593916],['g-233','g-656','g-157','g-165','g-104','c-97','c-34','g-628','g-204','g-633'])],[('72 * D2',[-0.50425,0.6124499999999999,-0.3384,-0.55875,-0.19215,-0.3538,-1.0505,-0.07465,0.24760000000000001,0.46425],[0.00021213203435594088,0.00035355339059331333,0.0007071067811865482,0.0013435028842544493,0.003181980515339447,0.005656854249492385,0.007778174593052108,0.008555992052357225,0.009475230867899753,0.010111626970967616],['g-700','g-511','g-123','c-24','g-676','g-79','g-203','c-64','g-152','c-46']),('48 * D2',[0.2359,0.4895,-0.23220000000000002,1.075,0.38634999999999997,0.0858,-0.14525,-0.3614,0.9309000000000001,-0.51],[0.0005656854249492542,0.0014142135623730963,0.0025455844122715854,0.0028284271247461927,0.003040559159102173,0.0035355339059327407,0.0045961940777125634,0.005091168824543131,0.005091168824543131,0.007636753236814657],['g-495','g-377','c-21','g-393','g-532','g-261','g-199','g-696','g-628','g-609']),('24 * D1',[-0.0022,-0.27859999999999996,0.3687,0.7812,-0.8389,0.0174,-0.55445,0.75665,-0.48365,-0.37539999999999996],[0.000848528137423857,0.0011313708498984691,0.0015556349186104294,0.0026870057685088986,0.0028284271247461927,0.0043840620433565946,0.005586143571373738,0.006293250352560286,0.011242997820866085,0.01244507934888324],['g-35','g-65','g-619','g-628','g-88','g-270','g-587','g-51','g-660','g-586']),('48 * D1',[0.9541999999999999,-1.0750000000000002,1.0550000000000002,0.5716,-0.02905,0.07885,0.72655,0.24345,0.02245,-0.27549999999999997],[0.0011313708498984299,0.0014142135623730963,0.0014142135623730963,0.0028284271247461927,0.0028991378028648445,0.003464823227814084,0.0034648232278140937,0.004030508652763309,0.005161879502661798,0.006222539674441601],['g-498','g-341','g-136','c-64','g-297','g-434','g-215','g-232','g-67','g-610'])],[('48 * D1',[-0.7063999999999999,0.3084,-1.1284999999999998,-0.6955,-0.13915,0.23355,-0.3683,-0.0465,-0.047700000000000006,-1.0665],[0.0005656854249492542,0.0007071067811865482,0.0007071067811866267,0.0028284271247461927,0.0031819805153394665,0.0036062445840513878,0.003676955262170035,0.003959797974644662,0.003959797974644667,0.004949747468305916],['g-468','g-669','g-621','g-606','g-747','g-108','g-677','g-724','c-15','g-545'])],[('48 * D1',[0.17205,-1.5065,0.3125,1.291,0.56195,-0.09695,-0.59785,-0.5928,0.095,-0.8223],[0.0003535533905932741,0.0007071067811864697,0.0014142135623730963,0.0014142135623730963,0.0016263455967291157,0.0017677669529663704,0.0020506096654409976,0.002404163056034311,0.005091168824543141,0.005656854249492385],['g-698','c-25','c-66','g-589','g-559','g-286','g-130','g-134','g-146','g-193'])],[('48 * D1',[1.3695,1.2345000000000002,0.18145,-0.718,0.88425,-0.4714,-0.33025,-2.274,0.6750499999999999,-5.1355],[0.0007071067811866267,0.0007071067811866267,0.0013435028842544493,0.0018384776310850566,0.0027577164466275456,0.003676955262170035,0.005020458146424484,0.005656854249492385,0.006293250352560286,0.006363961030678541],['g-133','g-99','c-80','c-45','g-377','g-345','c-74','g-8','g-125','g-59'])],[('48 * D1',[-0.25655,0.00445,-0.6892499999999999,-0.62755,-0.2554,0.64035,-0.67665,0.6067,-0.90935,-0.7637],[0.0004949747468306073,0.0006363961030678926,0.0017677669529664096,0.002192031021678291,0.0022627416997969383,0.0023334523779155853,0.0023334523779155853,0.002404163056034311,0.0027577164466275456,0.003676955262170035],['g-126','g-128','g-91','g-75','c-69','g-103','g-140','g-502','g-636','g-110'])],[('72 * D2',[0.28005,0.033,-0.1047,0.2349,0.0112,-0.2965,0.25505,0.6961999999999999,0.5502499999999999,-0.12334999999999999],[0.00021213203435598016,0.00042426406871193083,0.0011313708498984691,0.0032526911934581135,0.003676955262170047,0.004808326112068504,0.00601040764008562,0.007636753236814736,0.00799030662740797,0.008838834764831842],['g-740','c-93','g-22','g-220','g-606','g-224','g-298','g-223','g-597','g-1']),('24 * D2',[0.07615,0.4256,0.12605,0.83145,-0.8024,-0.16544999999999999,1.1705,0.20229999999999998,0.8013,0.38755],[7.071067811865677e-05,0.0009899494936611753,0.0027577164466275356,0.0044547727214752295,0.005232590180780503,0.006293250352560266,0.010606601717798144,0.010889444430272831,0.010889444430272888,0.012374368670764553],['g-486','g-39','c-55','g-477','g-168','g-420','g-666','g-1','g-414','g-144']),('48 * D2',[-0.14150000000000001,-0.37195,-0.00485,0.05605,-0.047850000000000004,-0.81915,0.40335,0.5802,0.08030000000000001,0.61505],[0.00028284271247460747,0.0014849242404917434,0.0016263455967290594,0.0020506096654409876,0.003040559159102158,0.0036062445840513878,0.004030508652763309,0.004101219330881916,0.006505382386916237,0.0072831998462214225],['g-303','g-373','g-128','g-396','c-19','g-437','g-272','g-150','g-619','g-541'])],[('48 * D1',[-0.26825,-1.134,0.8845000000000001,-0.9782500000000001,0.34415,0.41400000000000003,-0.2904,-1.0815000000000001,-0.6889000000000001,0.52475],[7.071067811864696e-05,0.0014142135623730963,0.0018384776310850566,0.0020506096654409976,0.004313351365237936,0.005232590180780464,0.00622253967444164,0.007778174593051951,0.008909545442950537,0.00982878425849295],['c-2','g-98','c-16','g-702','g-627','g-322','c-21','g-534','g-158','c-29'])],[('48 * D1',[0.3207,-0.2487,-0.084,0.7539,0.055900000000000005,0.6791,0.83405,-0.3359,0.44815,0.13774999999999998],[0.0005656854249492542,0.0011313708498984691,0.0014142135623730963,0.0016970562748476843,0.0018384776310850222,0.0024041630560342323,0.002616295090390173,0.0026870057685088595,0.0037476659402886814,0.004171930009000622],['c-83','g-264','c-75','g-360','g-145','g-679','g-7','g-539','c-49','c-54'])],[('48 * D1',[-0.13015,0.016399999999999998,0.3589,0.01545,0.1124,-0.32195,0.61565,-0.6589499999999999,0.6371,-0.2069],[0.0004949747468305876,0.0007071067811865457,0.0008485281374238421,0.0009192388155425111,0.0021213203435596446,0.0021920310216782916,0.005868986283848326,0.006717514421272168,0.0072124891681027755,0.007353910524340109],['g-290','g-505','g-30','c-46','c-38','g-227','g-308','g-197','g-348','g-216'])],[('48 * D1',[-0.4151,-1.0425,0.4304,2.7765,0.5358,-0.95865,0.39049999999999996,-0.23920000000000002,-0.8682000000000001,-1.6115],[0.0002828427124746271,0.0007071067811864697,0.0007071067811865482,0.0007071067811867837,0.0021213203435596446,0.002333452377915664,0.00311126983722082,0.0033941125496954275,0.0035355339059327407,0.003535533905932819],['g-6','g-712','g-448','g-146','g-514','g-753','g-266','g-549','g-125','c-80'])],[('48 * D1',[-0.2831,-0.25175000000000003,-0.34065,0.40054999999999996,0.41625,0.00745,-0.5995,0.02265,-0.16915,-0.27065],[0.0002828427124746271,0.000494974746830568,0.0014849242404917434,0.0017677669529663704,0.0017677669529663704,0.0020506096654409876,0.0026870057685088986,0.0027577164466275356,0.006434671708797581,0.007000357133746834],['g-15','g-48','g-320','g-473','g-76','g-588','g-120','g-464','g-56','g-197'])],[('48 * D1',[0.63565,1.6855,0.9224,0.4686,-0.80615,-0.9147000000000001,-0.8772500000000001,-0.21915,-1.3050000000000002,-0.4445],[0.0009192388155424891,0.002121320343559723,0.002404163056034311,0.0026870057685088595,0.0030405591591021333,0.0038183766184073284,0.0038890872965259755,0.005161879502661798,0.007071067811865481,0.0072124891681027755],['g-7','g-102','g-105','g-505','g-198','g-21','g-255','g-233','g-43','g-742'])],[('48 * D1',[-0.09625,0.03555,-0.26505,-0.0698,-0.7286,-0.49835,0.7330000000000001,-0.68465,0.21589999999999998,-0.43755],[0.00021213203435596054,0.0004949747468305827,0.0009192388155424891,0.00155563491861041,0.0016970562748476843,0.0020506096654409976,0.0035355339059327407,0.004030508652763348,0.005091168824543151,0.005586143571373738],['g-397','g-216','g-319','g-436','g-658','g-697','g-443','g-591','g-453','g-582'])],[('48 * D1',[0.26215,-1.4675,1.1055000000000001,0.45455,0.6456500000000001,-0.04035,0.36639999999999995,0.3276,-0.39955,-1.303],[0.0004949747468306073,0.0007071067811864697,0.002121320343559566,0.0033234018715767605,0.006010407640085699,0.006576093065034889,0.008909545442950499,0.009192388155425127,0.009404520189781067,0.009899494936611674],['g-254','g-395','g-392','g-538','g-264','g-564','g-55','g-334','g-660','c-74'])],[('48 * D1',[1.0955,0.08449999999999999,1.0270000000000001,0.037099999999999994,1.3035,-2.2435,-1.2125,-0.73195,1.00285,2.9425],[0.0007071067811866267,0.0008485281374238519,0.0014142135623730963,0.001555634918610405,0.003535533905932819,0.006363961030679169,0.007778174593051951,0.00841457069611993,0.00869741340859444,0.009192388155425047],['g-180','g-555','g-41','g-624','g-501','g-671','g-726','g-419','g-10','c-66'])],[('48 * D1',[0.23425,-0.21939999999999998,-0.05495,0.28325,0.4548,0.1256,-0.48095,-1.1955,-0.14285,0.17830000000000001],[0.0009192388155425087,0.0009899494936611555,0.001909188309203679,0.0028991378028648397,0.0032526911934581135,0.004101219330881975,0.0045961940777125634,0.004949747468305916,0.005303300858899111,0.0057982756057296985],['g-286','g-361','g-541','g-23','g-269','c-66','g-543','g-129','g-727','g-360'])],[('48 * D1',[-0.77015,0.6657,-0.19725,0.64895,0.37795,1.1244999999999998,-0.2025,-0.12755,0.26915,0.5892999999999999],[7.071067811864696e-05,0.00028284271247466634,0.0007778174593051951,0.0010606601717798615,0.0020506096654409976,0.002121320343559566,0.0025455844122715854,0.0026162950903902125,0.0028991378028648397,0.004949747468305837],['g-208','c-81','g-201','c-63','c-30','g-670','g-541','g-229','g-599','g-12'])],[('72 * D1',[0.75225,0.36145000000000005,-0.09275,0.5038,0.95035,0.2138,0.31965,-0.18,0.0,0.27285],[0.00035355339059323483,0.0017677669529663704,0.0037476659402887014,0.003959797974644701,0.004171930009000642,0.004242640687119289,0.0044547727214752295,0.004949747468305837,0.00565685424949238,0.005727564927611032],['g-210','g-724','g-611','g-323','g-612','g-31','g-433','g-395','g-117','g-186']),('72 * D2',[0.07594999999999999,-0.23215,0.44315000000000004,0.3156,0.6617,-0.32315,-0.022,-0.57915,-0.75345,-0.2169],[0.000777817459305205,0.0010606601717798223,0.0010606601717798223,0.0022627416997969383,0.0028284271247461927,0.004313351365237936,0.006081118318204309,0.009970205614730322,0.011950104602052673,0.012869343417595162],['g-197','g-428','g-627','g-688','g-251','g-763','g-262','g-736','g-82','g-550'])],[('48 * D1',[0.03745,-0.2743,0.54615,-0.6976500000000001,-0.81,-0.2029,-0.3204,0.5057,-0.9269000000000001,-0.10515],[0.0003535533905932741,0.0009899494936611753,0.0016263455967290372,0.0020506096654409976,0.0035355339059327407,0.003959797974644662,0.0059396969619670125,0.007071067811865481,0.00820243866176391,0.009121677477306469],['g-95','g-407','c-12','c-3','c-10','g-126','g-642','g-426','c-44','g-11'])],[('48 * D1',[-0.0446,1.2765,-0.34919999999999995,0.9432,0.14795,-0.6919500000000001,-0.9274,0.021249999999999998,0.2336,-2.3205],[0.00042426406871193083,0.002121320343559723,0.0024041630560342714,0.003252691193458153,0.0034648232278140937,0.0036062445840513878,0.003818376618407407,0.0050204581464244865,0.005232590180780464,0.006363961030679169],['g-221','g-82','g-662','g-348','g-220','g-226','c-39','g-694','g-617','g-389'])],[('48 * D1',[0.88045,0.2412,0.33065,0.0838,0.5103500000000001,0.478,-0.23035,0.023899999999999998,0.68215,0.523],[0.00021213203435601938,0.00042426406871192107,0.0012020815280171554,0.00155563491861041,0.0017677669529664096,0.0029698484809835258,0.004454772721475249,0.004525483399593903,0.0047376154339498966,0.0059396969619669735],['g-17','g-335','g-267','g-433','g-438','g-614','g-282','g-63','g-117','g-138'])]]
    c = 0
    r=150
    N = 10
    for li_ in li:
        c += 1
        indices = []
        for td, m, s, cols in li_:
            tmp = df[df["time_dose"] == td][cols]
            for i in range(N):
                tmp = tmp[(m[i] - r*s[i] < tmp[cols[i]]) & (tmp[cols[i]] < m[i] + r*s[i])]
            indices += tmp.index.to_list()
        name = "onehot{}".format(c)
        df[name] = 0
        df[name].iloc[indices] = 1
    df1 = df[:size]
    df2 = df[size:].reset_index(drop=True)
    return df1, df2
        

seeds = [0, 1, 2, 3, 4, 5, 6]
SCALE = "quantile"

    

In [3]:
# g772, c100, 206クラス、402クラスの分類

train_df = pd.read_csv("../../../Data/Raw/train_features.csv")
test_df = pd.read_csv("../../../Data/Raw/test_features.csv")
pub_test_df = pd.read_csv("../../../Data/Raw/test_features.csv")
drug_df = pd.read_csv("../../../Data/Raw/train_drug.csv")#

train_df = train_df.merge(drug_df, on="sig_id")

y = pd.read_csv("../../../Data/Raw/train_targets_scored.csv")
y_non = pd.read_csv("../../../Data/Raw/train_targets_nonscored.csv")
y_all = pd.concat([y, y_non.drop("sig_id", axis=1)], axis=1)
y = y.merge(drug_df, on='sig_id', how='left') #

GENES = [col for col in train_df.columns if col.startswith("g-")]
CELLS = [col for col in train_df.columns if col.startswith("c-")]
BIOS = GENES + CELLS


SCORED_MOAS = [col for col in y.columns if col != "sig_id" and col != "drug_id"]#
NONSCORED_MOAS = [col for col in y_non.columns if col != "sig_id"]
ALL_MOAS = SCORED_MOAS + NONSCORED_MOAS


TR_SIZE = train_df.shape[0]
TE_SIZE = test_df.shape[0]

train_nonvehicle_index = train_df[train_df["cp_type"] != "ctl_vehicle"].index
test_nonvehicle_index = test_df[test_df["cp_type"] != "ctl_vehicle"].index

train_df["time_dose"] = train_df["cp_time"].astype(str) + " * " + train_df["cp_dose"]
test_df["time_dose"] = test_df["cp_time"].astype(str) + " * " + test_df["cp_dose"]
pub_test_df["time_dose"] = pub_test_df["cp_time"].astype(str) + " * " + pub_test_df["cp_dose"]

# remove cp_type = ctl_vehicle
mask = train_df["cp_type"] != "ctl_vehicle"
train_df = train_df[mask].drop("cp_type", axis=1).reset_index(drop=True)
test_df = test_df[test_df["cp_type"] != "ctl_vehicle"].drop("cp_type", axis=1).reset_index(drop=True)
pub_test_df = pub_test_df[pub_test_df["cp_type"] != "ctl_vehicle"].drop("cp_type", axis=1).reset_index(drop=True)
y_nonv = y[mask].reset_index(drop=True)#

scored = y_nonv.copy()#
y_nonv.drop("drug_id", axis=1, inplace=True)#
y.drop("drug_id", axis=1, inplace=True)#

TR_NONV_SIZE = train_df.shape[0]
TE_NONV_SHAPE = test_df.shape[0]

In [4]:
train_df, test_df = extract(train_df, test_df)
test_df.drop("drug_id", axis=1, inplace=True)
ONEHOT = [col for col in train_df if col.startswith("onehot")]

In [5]:
test_df[ONEHOT].sum().sum()

14

In [6]:
#prod_cols = [    ['g-712',  'g-208',  'g-38',  'g-100',  'g-123',  'g-328',  'g-744',  'g-248',  'g-460',  'g-731',  'g-417',  'g-349',  'g-131'], ['g-228',  'g-75',  'g-67',  'g-760',  'g-37',  'g-406',  'g-50',  'g-672',  'g-63',  'g-72',  'g-195'], ['g-100', 'g-157', 'g-178'],['g-183', 'g-300', 'g-767'],['g-50', 'g-37', 'g-489', 'g-257', 'g-332'],['g-270', 'g-135', 'g-231', 'g-158', 'g-478', 'g-146', 'g-491', 'g-392'],['g-745', 'g-635', 'g-235'], ['g-300', 'g-414', 'g-62', 'g-34'], ['g-91', 'g-392'], ['g-75', 'g-113'], ['g-599', 'g-261', 'g-38', 'g-146', 'g-392', 'g-512', 'g-744'], ['g-50', 'g-332', 'g-37', 'g-58', 'g-705'], ['g-157', 'g-178'],['g-759', 'g-100', 'g-167', 'g-75', 'g-431', 'g-189', 'g-522', 'g-91'],['g-202', 'g-385', 'g-769'],           ]
prod_cols = [['g-145', 'g-201', 'g-208'], ['g-370', 'g-508', 'g-37'], ['g-38', 'g-392', 'g-707'], ['g-328', 'g-28', 'g-392'], ['g-441', 'g-157', 'g-392'], ['g-181', 'g-100', 'g-392'], ['g-67', 'g-760', 'g-50'], ['g-731', 'g-100', 'g-707'], ['g-478', 'g-468', 'g-310'], ['g-91', 'g-145', 'g-208'], ['g-106', 'g-744', 'g-91'], ['g-131', 'g-208', 'g-392'], ['g-144', 'g-123', 'g-86'], ['g-228', 'g-72', 'g-67'], ['g-31', 'g-328', 'g-460'], ['g-392', 'g-731', 'g-100'], ['g-732', 'g-744', 'g-707'], ['g-705', 'g-375', 'g-704'], ['g-508', 'g-50', 'g-411'], ['g-234', 'g-58', 'g-520'], ['g-503', 'g-761', 'g-50'], ['g-113', 'g-75', 'g-178'], ['g-50', 'g-508', 'g-113'], ['g-113', 'g-375', 'g-75'], ['g-576', 'g-452', 'g-392'], ['g-50', 'g-37', 'g-36'], ['g-707', 'g-133', 'g-392'], ['g-484', 'g-392', 'g-544'], ['g-508', 'g-67', 'g-370'], ['g-123', 'g-731', 'g-100'], ['g-298', 'g-477', 'g-644'], ['g-72', 'g-370', 'g-50'], ['g-67', 'g-178', 'g-113'], ['g-744', 'g-608', 'g-100'], ['g-91', 'g-100', 'g-707'], ['g-37', 'g-228', 'g-202'], ['g-37', 'g-300', 'g-370'], ['g-234', 'g-508', 'g-595'], ['g-596', 'g-744', 'g-707'], ['g-300', 'g-227', 'g-591'], ['g-135', 'g-392', 'g-512'], ['g-731', 'g-744', 'g-158'], ['g-69', 'g-707', 'g-100'], ['g-276', 'g-653', 'g-291'], ['g-624', 'g-615', 'g-189'], ['g-181', 'g-707', 'g-38'], ['g-72', 'g-75', 'g-508'], ['g-231', 'g-707', 'g-392'], ['g-508', 'g-37', 'g-72'], ['g-725', 'g-712', 'g-640'], ['g-67', 'g-644', 'g-113'], ['g-508', 'g-228', 'g-656'], ['g-185', 'g-37', 'g-672'], ['g-370', 'g-50', 'g-503'], ['g-201', 'g-745', 'g-599'], ['g-332', 'g-50', 'g-571'], ['g-50', 'g-37', 'g-59'], ['g-508', 'g-113', 'g-231'], ['g-707', 'g-158', 'g-100'], ['g-257', 'g-50', 'g-72']]

for cols in prod_cols:
    name = "prod-" + " * ".join(cols)
    train_df[name] = train_df[cols].mean(axis=1)
    test_df[name] = test_df[cols].mean(axis=1)
    pub_test_df[name] = pub_test_df[cols].mean(axis=1)

PRODS = [col for col in train_df.columns if col.startswith("prod-")]

In [7]:
#out fold preprocessing

#variance threshold
VAR_THRESHOLD = 0.8
drop_cols = []
temp = pd.concat([train_df, pub_test_df])
for col in BIOS+PRODS:
    if temp[col].var() <= VAR_THRESHOLD:
        drop_cols.append(col)

print("drop cols num : {}".format(len(drop_cols)))
train_df.drop(columns=drop_cols, inplace=True)
test_df.drop(columns=drop_cols, inplace=True)
pub_test_df.drop(columns=drop_cols, inplace=True)

GENES_ = [col for col in train_df.columns if col.startswith("g-")]
CELLS_ = [col for col in train_df.columns if col.startswith("c-")]
BIOS_ = GENES_ + CELLS_
PRODS = [col for col in train_df.columns if col.startswith("prod-")]
        
del temp




# onehot encode of categorical feature and drop
drop_cols = ["cp_time", "cp_dose", "time_dose"]
train_df = pd.concat([pd.get_dummies(train_df["time_dose"], prefix="onehot", drop_first=True), train_df.drop(drop_cols, axis=1) ], axis=1)
test_df = pd.concat([pd.get_dummies(test_df["time_dose"], prefix="onehot", drop_first=True), test_df.drop(drop_cols, axis=1) ], axis=1)
pub_test_df = pd.concat([pd.get_dummies(pub_test_df["time_dose"], prefix="onehot", drop_first=True), pub_test_df.drop(drop_cols, axis=1) ], axis=1)


# aggregation feature
print("agg")
"""
for df in [train_df, pub_test_df, test_df]:
    df["agg-sum-g"] = (df[GENES_].sum(axis=1) - df[GENES_].sum(axis=1).min() + 1).map(np.log)
    df["agg-mean-g"] = (df[GENES_].mean(axis=1) - df[GENES_].mean(axis=1).min() + 1).map(np.log)
    df["agg-std-g"] = df[GENES_].std(axis=1).map(np.log)
    df["agg-kurt-g"] = (df[GENES_].kurt(axis=1) - df[GENES_].kurt(axis=1).min() + 1).map(np.log)
    df["agg-skew-g"] = (df[GENES_].skew(axis=1) - df[GENES_].skew(axis=1).min() + 1).map(np.log)
    df["agg-sum-c"] = (df[CELLS_].sum(axis=1) - df[CELLS_].sum(axis=1).min() +1).map(np.log)
    df["agg-mean-c"] = (df[CELLS_].mean(axis=1) - df[CELLS_].mean(axis=1).min() +1).map(np.log)
    df["agg-std-c"] = df[CELLS_].std(axis=1).map(np.log)
    df["agg-kurt-c"] = (df[CELLS_].kurt(axis=1) - df[CELLS_].kurt(axis=1).min() +1).map(np.log)
    df["agg-skew-c"] = (df[CELLS_].skew(axis=1) - df[CELLS_].skew(axis=1).min() +1).map(np.log)
    df["agg-sum-gc"] = (df[BIOS_].sum(axis=1) - df[BIOS_].sum(axis=1).min() + 1).map(np.log)
    df["agg-mean-gc"] = (df[BIOS_].mean(axis=1) - df[BIOS_].mean(axis=1).min() + 1).map(np.log)
    df["agg-std-gc"] = df[BIOS_].std(axis=1).map(np.log)
    df["agg-kurt-gc"] = (df[BIOS_].kurt(axis=1) - df[BIOS_].kurt(axis=1).min() + 1).map(np.log)
    df["agg-skew-gc"] = (df[BIOS_].skew(axis=1) - df[BIOS_].skew(axis=1).min() + 1).map(np.log)
"""

for df in [train_df, pub_test_df, test_df]:
    df["agg-sum-g"] = df[GENES_].sum(axis=1)
    df["agg-mean-g"] = df[GENES_].mean(axis=1)
    df["agg-std-g"] = df[GENES_].std(axis=1)
    df["agg-kurt-g"] = df[GENES_].kurt(axis=1)
    df["agg-skew-g"] = df[GENES_].skew(axis=1)
    df["agg-sum-c"] = df[CELLS_].sum(axis=1)
    df["agg-mean-c"] = df[CELLS_].mean(axis=1)
    df["agg-std-c"] = df[CELLS_].std(axis=1)
    df["agg-kurt-c"] = df[CELLS_].kurt(axis=1)
    df["agg-skew-c"] = df[CELLS_].skew(axis=1)
    df["agg-sum-gc"] = df[BIOS_].sum(axis=1)
    df["agg-mean-gc"] = df[BIOS_].mean(axis=1)
    df["agg-std-gc"] = df[BIOS_].std(axis=1)
    df["agg-kurt-gc"] = df[BIOS_].kurt(axis=1)
    df["agg-skew-gc"] = df[BIOS_].skew(axis=1)

AGG = [col for col in train_df.columns if col.startswith("agg-")]

drop cols num : 67
agg


In [8]:
"""print("kmeans clustering")
n_g_clusters = 22
n_c_clusters = 5
kmeans_g = KMeans(n_clusters=n_g_clusters, random_state=42).fit(train_df.append(pub_test_df)[GENES_])  
for df in [train_df, test_df, pub_test_df]:
    df["kmeans"] = kmeans_g.predict(df[GENES_])

train_df = pd.concat([pd.get_dummies(train_df["kmeans"], prefix="kmeans-g-", drop_first=True), train_df.drop("kmeans", axis=1) ], axis=1)
test_df = pd.concat([pd.get_dummies(test_df["kmeans"], prefix="kmeans-g-", drop_first=True), test_df.drop("kmeans", axis=1) ], axis=1)
pub_test_df = pd.concat([pd.get_dummies(pub_test_df["kmeans"], prefix="kmeans-g-", drop_first=True), pub_test_df.drop("kmeans", axis=1) ], axis=1)

kmeans_c = KMeans(n_clusters=n_c_clusters, random_state=42).fit(train_df.append(pub_test_df)[CELLS_])  
for df in [train_df, test_df, pub_test_df]:
    df["kmeans"] = kmeans_c.predict(df[CELLS_])

train_df = pd.concat([pd.get_dummies(train_df["kmeans"], prefix="kmeans-c-", drop_first=True), train_df.drop("kmeans", axis=1) ], axis=1)
test_df = pd.concat([pd.get_dummies(test_df["kmeans"], prefix="kmeans-c-", drop_first=True), test_df.drop("kmeans", axis=1) ], axis=1)
pub_test_df = pd.concat([pd.get_dummies(pub_test_df["kmeans"], prefix="kmeans-c-", drop_first=True), pub_test_df.drop("kmeans", axis=1) ], axis=1)"""

'print("kmeans clustering")\nn_g_clusters = 22\nn_c_clusters = 5\nkmeans_g = KMeans(n_clusters=n_g_clusters, random_state=42).fit(train_df.append(pub_test_df)[GENES_])  \nfor df in [train_df, test_df, pub_test_df]:\n    df["kmeans"] = kmeans_g.predict(df[GENES_])\n\ntrain_df = pd.concat([pd.get_dummies(train_df["kmeans"], prefix="kmeans-g-", drop_first=True), train_df.drop("kmeans", axis=1) ], axis=1)\ntest_df = pd.concat([pd.get_dummies(test_df["kmeans"], prefix="kmeans-g-", drop_first=True), test_df.drop("kmeans", axis=1) ], axis=1)\npub_test_df = pd.concat([pd.get_dummies(pub_test_df["kmeans"], prefix="kmeans-g-", drop_first=True), pub_test_df.drop("kmeans", axis=1) ], axis=1)\n\nkmeans_c = KMeans(n_clusters=n_c_clusters, random_state=42).fit(train_df.append(pub_test_df)[CELLS_])  \nfor df in [train_df, test_df, pub_test_df]:\n    df["kmeans"] = kmeans_c.predict(df[CELLS_])\n\ntrain_df = pd.concat([pd.get_dummies(train_df["kmeans"], prefix="kmeans-c-", drop_first=True), train_df.dro

In [9]:
X = train_df.drop("sig_id", axis=1)
y_nonv = y_nonv.drop("sig_id", axis=1).values
y = y.drop("sig_id", axis=1).values

## Dateset Class

In [10]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

## func 

In [11]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        # if cycle
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)


        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss


def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

## Model

In [12]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets):
        super(Model, self).__init__()
        self.hidden1 = 2048
        self.hidden2 =  1024
        
        self.nn_model = nn.Sequential(
            nn.BatchNorm1d(num_features),
            nn.utils.weight_norm(nn.Linear(num_features, self.hidden1)),
            nn.LeakyReLU(),
            
            nn.BatchNorm1d(self.hidden1),
            nn.Dropout(0.25),
            nn.utils.weight_norm(nn.Linear(self.hidden1, self.hidden2)),
            nn.LeakyReLU(),
            
            nn.BatchNorm1d(self.hidden2),
            nn.Dropout(0.25),
            nn.utils.weight_norm(nn.Linear(self.hidden2, num_targets)),
        )

    
    def forward(self, x):
        return self.nn_model(x)


## run train

In [13]:
def run_training(model, trainloader, validloader, epoch_, optimizer, scheduler, loss_fn, loss_tr, early_stopping_steps, verbose, device, fold, seed):
    
    early_step = 0
    best_loss = np.inf
    best_epoch = 0
    
    start = time.time()
    t = time.time() - start
    for epoch in range(epoch_):
        train_loss = train_fn(model, optimizer, scheduler, loss_tr, trainloader, device)
        valid_loss = valid_fn(model, loss_fn, validloader, device)
        if epoch % verbose==0 or epoch==epoch_-1:
            t = time.time() - start
            print(f"EPOCH: {epoch}, train_loss: {train_loss}, valid_loss: {valid_loss}, time: {t}")
        
        if valid_loss < best_loss:
            best_loss = valid_loss
            torch.save(model.state_dict(), 'dnn_weights2/{}_{}.pt'.format(seed, fold))
            early_step = 0
            best_epoch = epoch
        
        elif early_stopping_steps != 0:
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                t = time.time() - start
                print(f"early stopping in iteration {epoch},  : best itaration is {best_epoch}, valid loss is {best_loss}, time: {t}")
                return model
    t = time.time() - start       
    print(f"training until max epoch {epoch_},  : best itaration is {best_epoch}, valid loss is {best_loss}, time: {t}")
    return model
            
    
def predict(model, testloader, device):
    model.to(device)
    predictions = inference_fn(model, testloader, device)
    
    return predictions

In [14]:
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

## Training by Fold

In [15]:
BATCH_SIZE = 128
DEVICE = ('cuda:2' if torch.cuda.is_available() else 'cpu')
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
EPOCHS = 25
EARLY_STOPPING_STEPS = 10

train_preds = np.zeros((X.shape[0], y_nonv.shape[1]))
preds = np.zeros((test_df.shape[0], y_nonv.shape[1]))
imps = []
imp_cols = []
folds = []
test_cv_preds = []

for seed in seeds:
    seed_everything(seed)
    K = 5
    kf = MultilabelStratifiedKFold(n_splits=K, random_state=seed, shuffle=True)
    train_pred = np.zeros(train_preds.shape)
    
    
    ###############################################################################################
    # LOAD LIBRARIES
    targets = SCORED_MOAS.copy()

    # LOCATE DRUGS
    vc = scored["drug_id"].value_counts()
    vc1 = vc.loc[vc<=18].index.sort_values()
    vc2 = vc.loc[vc>18].index.sort_values()

    # STRATIFY DRUGS 18X OR LESS
    dct1 = {}; dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
    tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.index[idxV].values} # drug id がどのフォールドに属すか格納
        dct1.update(dd)

    # STRATIFY DRUGS MORE THAN 18X
    skf = MultilabelStratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
    tmp = scored.loc[scored["drug_id"].isin(vc2)].reset_index(drop=True)
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp["sig_id"][idxV].values}
        dct2.update(dd)

    # ASSIGN K
    scored['fold'] = scored.drug_id.map(dct1)
    scored.loc[scored["fold"].isna(),'fold'] = scored.loc[scored["fold"].isna(),'sig_id'].map(dct2)
    scored["fold"] = scored["fold"].astype('int8')
    ###############################################################################################

    #for fold, (train_index, valid_index) in enumerate(kf.split(X, y_nonv)):    
    for fold in range(K):
        train_index = scored[scored["fold"] != fold].index.to_list()
        valid_index = scored[scored["fold"] == fold].index.to_list()
        print("======================== fold {} ========================".format(fold+1))
        folds.append(train_index)
                
        ### split data ##########
        train_X = X.iloc[train_index]
        train_y = y_nonv[train_index]
        valid_X = X.iloc[valid_index]
        valid_y = y_nonv[valid_index]
        test_X = (test_df.drop("sig_id", axis=1))
        pub_test_X = (pub_test_df.drop("sig_id", axis=1))

        
    
        train_X.drop("drug_id", axis=1, inplace=True)
        valid_X.drop("drug_id", axis=1, inplace=True)
        
        ### scaler ##########
        print(SCALE)
        scale_cols = BIOS_#+PRODS+AGG
        scaler = make_scaler(SCALE, seed).fit(train_X.append(pub_test_X)[scale_cols])
        for df in [train_X, valid_X, test_X, pub_test_X]:
            df[scale_cols] = scaler.transform(df[scale_cols])

        



        ### PCA ##########
        print("PCA")
        n_decom_g = 80
        n_decom_c = 10
        decom_g_cols = [f"pca_g-{i}" for i in range(n_decom_g)]
        decom_c_cols = [f"pca_c-{i}" for i in range(n_decom_c)]
        
        pca_g = PCA(n_components = n_decom_g, random_state = seed).fit(train_X.append(pub_test_X)[GENES_])
        pca_c = PCA(n_components = n_decom_c, random_state = seed).fit(train_X.append(pub_test_X)[CELLS_])
        for df in [train_X, valid_X, test_X, pub_test_X]:
            df[decom_g_cols] = pca_g.transform(df[GENES_])
            df[decom_c_cols] = pca_c.transform(df[CELLS_])

        

        # prepare data for training
        train_X = train_X.values
        valid_X = valid_X.values
        test_X = test_X.values
        print(X.shape)
        
        
        # ================================model training===========================
        train_dataset = MoADataset(train_X, train_y)
        valid_dataset = MoADataset(valid_X, valid_y)
        test_dataset = TestDataset(test_X)
        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
        validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
        testloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

        model = Model(
            num_features=train_X.shape[1],
            num_targets=train_y.shape[1],
        )

        model.to(DEVICE)
        
        
        optimizer = torch.optim.Adam( model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY,)
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader) )
        
        loss_fn = nn.BCEWithLogitsLoss()
        loss_tr = SmoothBCEwLogits(smoothing=1e-3)
        
        # train
        model = run_training(
            model=model,
            trainloader=trainloader,
            validloader=validloader,
            epoch_=EPOCHS,
            optimizer=optimizer,
            scheduler=scheduler,
            loss_fn=loss_fn,
            loss_tr=loss_tr,
            early_stopping_steps=EARLY_STOPPING_STEPS,
            device=DEVICE,
            verbose=5,
            fold=fold,
            seed=seed,)
        model.load_state_dict(torch.load('dnn_weights2/{}_{}.pt'.format(seed, fold)), DEVICE)
        
        #valid predict
        val_preds = predict(
            model=model,
            testloader=validloader,
            device=DEVICE,)
        
        #test predict
        test_preds = predict(
            model=model,
            testloader=testloader,
            device=DEVICE)
        
        # ================================model training===========================

        train_pred[valid_index] +=  val_preds
        
        preds += test_preds / (K*len(seeds))

    print("seed {} , cv score : {}".format(seed, metric(y_nonv, train_pred)))
    train_preds += train_pred/len(seeds)
print("cv score : {}".format(metric(y_nonv, train_preds)))

======================== fold 1 ========================
quantile
PCA
(21948, 1080)
EPOCH: 0, train_loss: 0.5030722723113022, valid_loss: 0.031159654153244835, time: 2.2856056690216064
EPOCH: 5, train_loss: 0.020261116717280685, valid_loss: 0.01829945570124047, time: 13.36126184463501
EPOCH: 10, train_loss: 0.02023822761586179, valid_loss: 0.018278853222727775, time: 24.516051769256592
EPOCH: 15, train_loss: 0.019682485718226086, valid_loss: 0.017618682608008383, time: 35.00325798988342
EPOCH: 20, train_loss: 0.018150919650181913, valid_loss: 0.017542051790016038, time: 45.82145309448242
EPOCH: 24, train_loss: 0.01671665222586497, valid_loss: 0.017467049642332963, time: 55.15932106971741
training until max epoch 25,  : best itaration is 19, valid loss is 0.017387905344367027, time: 55.16050863265991
======================== fold 2 ========================
quantile
PCA
(21948, 1080)
EPOCH: 0, train_loss: 0.5022387573414523, valid_loss: 0.028393395683344674, time: 2.0513365268707275
EPOC

KeyboardInterrupt: 

In [ ]:
train_preds2 = np.zeros((TR_SIZE,  y.shape[1]))
train_preds2[train_nonvehicle_index] = train_preds


preds2 = np.zeros((TE_SIZE, y.shape[1]))
preds2[test_nonvehicle_index] = preds

print("cv score : {}".format(metric(y, train_preds2)))

In [ ]:
sub_df = pd.read_csv("../../../Data/Raw/sample_submission.csv")
#sub_df = pd.read_csv("../input/lish-moa/sample_submission.csv")
cols = [col for col in sub_df.columns if col != "sig_id"]
sub_df[cols] = preds2
sub_df.to_csv("submission.csv", index=False)

In [ ]:
train_sub = pd.read_csv("../../../Data/Raw/train_targets_scored.csv")
cols = [col for col in train_sub.columns if col != "sig_id"]
train_sub[cols] = train_preds2
train_sub.to_csv("train_preds.csv", index=False)

In [ ]:
!zip dnn_weights2.zip dnn_weights2/*

In [ ]:
train_df[ONEHOT].sum()

In [ ]:
test_df[ONEHOT].sum()

In [ ]:
test_df